In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import os
# from compass_smartsheet import CompassSmartsheet
# from ftplib import FTP
import cisco_fiscal_calendar as fc
import hashlib

In [2]:
pd.__version__

'1.2.2'

In [3]:
today_timestamp = dt.datetime.now().isoformat()[:19]

In [4]:
def convert_nan_to_int_to_str(dataframe_cell):
        if pd.isnull(dataframe_cell):
            return pd.NA
        else:
            try:
                return str(int(dataframe_cell))
            except:
                return pd.NA

In [5]:
# def convert_requestid_to_str(requestid_cell):
#     return str(int(requestid_cell))

### Prepare the Partners Dictionary

In [6]:
partner = pd.read_excel(r'C:\Users\phsheari\Documents\Compass Partners\Compass Partners Hash.xlsx')

In [7]:
compass_partner_names = partner['CompassPartnerName']
standardname = partner['StandardName']

In [8]:
partner_dict = {}
for k,v in zip(compass_partner_names,standardname):
    partner_dict[k] = v

In [9]:
# partner_dict

In [10]:
sss_map = pd.read_excel(r'C:\Users\phsheari\Documents\Francisco\SSS_GEO Mapping.xlsx')

In [11]:
sss = list(sss_map['SSS'])

In [12]:
geo = list(sss_map['SL3'])

In [13]:
sssgeo_dict = {}
for k, v in zip(sss,geo):
    sssgeo_dict[k] = v

In [14]:
# sssgeo_dict

#### Bring in FINBI Accrued Results to verify booked records are tagged as '5 - Closed Won'

In [15]:
finbi_data_types = {'Sales Level 1':str,                     
                    'Sales Level 2':str, 
                    'Sales Level 3':str, 
                    'CAV ID':str, 
                    'CAV Name':str,
                    'GUID':str, 
                    'GUID Name':str, 
                    'Partner':str,
                    'Booked Deal ID':str, 
                    'CX Product Category':str, 
                    'Fiscal Year':str,
                    'Fiscal Quarter ID':str, 
                    'Fiscal Week ID':str, 
                    'Fiscal Week of Year':str, 
                    'ReportWeek':str, 
                    'ProductType':str,
                    'TCV':np.float32, 
                    'ACV':np.float32, 
                    'Request ID':str, 
                    'Readout_Date':np.datetime64, 
                    'Campaign Type':str,
                    'Campaign':str, 
                    'Target Campaign Flag':str, 
                    'Partner Deal':str,
                    'Service Source':str,
                    'Compass GUID Name':str,
                    'Compass Deal ID':str,
                    'Compass Advisor':str,
                    'Readout Delivery Status':str,
                    'Items Recommended':str,}

In [16]:
finbi_path = r'C:\Users\phsheari\Documents\FIN BI Data\FINBI_Bookings_Accrued_Results.xlsx'
finbi_sheetname = 'YTD Transactions'
finbi_columns = ['Sales Level 1', 'Sales Level 2', 'Sales Level 3', 'CAV ID', 'CAV Name',
       'GUID', 'GUID Name', 'Partner','Booked Deal ID', 'CX Product Category', 'Fiscal Year',
       'Fiscal Quarter ID', 'Fiscal Week ID', 'Fiscal Week of Year', 'ReportWeek', 'ProductType',
       'TCV', 'ACV', 'Request ID', 'Readout_Date', 'Campaign Type',
       'Campaign', 'Target Campaign Flag', 'Partner Deal','Service Source','Compass GUID Name','Compass Deal ID','Compass Advisor','Readout Delivery Status','Items Recommended', ]
finbi_df = pd.read_excel(finbi_path, sheet_name = finbi_sheetname, usecols=finbi_columns, dtype=finbi_data_types)
finbi_df['Request ID'] = finbi_df['Request ID'].apply(convert_nan_to_int_to_str)
finbi_df['Readout_Date'] = finbi_df['Readout_Date'].apply(lambda x: pd.to_datetime(x).date() if x != pd.NA else x)

finbi_df.to_pickle(r'C:\Users\phsheari\Documents\FIN BI Data\FINBI_Bookings_Accrued_Results.pkl')
finbi_df.to_excel(r'C:\Users\phsheari\Documents\FIN BI Data\FINBI_Bookings_Accrued_Results_Copy.xlsx', index=False)

#maven_fields_to_drop = ['Ind','Compass Exists?','Compass Claim?','Notes','Respondant_File','AddIncre']
# maven_version = finbi_df #.drop(columns=maven_fields_to_drop)
# maven_version.to_excel(r'C:\Users\phsheari\Documents\FIN BI Data\Maven\Compass Bookings.xlsx', index=False)

finbi_requests = list(finbi_df.loc[finbi_df['Request ID'].notnull()]['Request ID'].unique())

In [17]:
# finbi_df.columns
finbi_requests.sort()

In [18]:
# finbi_requests

### Start the pull from SMARTSHEET

In [19]:
start = dt.datetime.now()

# # The Compass Smartsheet object init's a class which retrieves, cleans, & modifies records to prepare for presentation. 
# # It also unmarks, then remarks the Duplicate DealID Request records

# dflist = []
# for i in range(1,4,1):                  # This paginates the CompassSmartsheet Object, pulling 10k records per page
#     cs = CompassSmartsheet(page = i)
#     tmpdf = cs.get_report_dataframe()   # this takes the dataframe object from cs and stores it in tmpdf
#     dflist.append(tmpdf)                # appends each tmpdf to dflist, for future concatenation
    
# nonbooked = pd.concat(dflist)

# finish = dt.datetime.now()
# elapsed = (finish - start)
# print(f'Compass object was created in {elapsed} minutes. It shows {nonbooked.shape[0]} rows in the dataframe.')

In [20]:
nonbooked = pd.read_pickle(r'C:\Users\phsheari\Documents\Compass SNOWFLAKE\data_download\result.pkl')

In [21]:
today_file = today_timestamp.replace('-','').replace('T','_').replace(':','')

In [22]:
today_file

'20220419_133613'

In [23]:
nonbooked.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42717 entries, 0 to 42716
Columns: 107 entries, Request ID to Readout_Date
dtypes: bool(9), datetime64[ns](7), float64(12), int64(16), object(63)
memory usage: 32.3+ MB


### Push Booked deals to a reference file for corrective measures

In [24]:
nickname_files_path = r'C:\Users\phsheari\Documents\Compass Nickname'
nonbooked['Nickname'] = pd.NA
USECOLS = ['Request ID']
DTYPES = {'Request ID':str}

entries = os.listdir(nickname_files_path)
for f_name in entries:
    tempdf = pd.read_excel(nickname_files_path + '\\' + f_name, usecols = USECOLS, dtype=DTYPES)
    id_list = list(tempdf['Request ID'].unique())
    nonbooked.loc[nonbooked['Request ID'].isin(id_list),'Nickname'] = f_name[:-5]
    print(f'Nickname is {f_name[:-5]} and its count is {len(id_list)}')

Nickname is Mirthas Top68 and its count is 52
Nickname is US Commercial Missed Attach Mari 20211130 and its count is 376


In [25]:
nonbooked.loc[nonbooked['Nickname'].notnull()][['Request ID','Create Date','Customer Name','Campaign','Nickname','WORKING STATUS']]

,Request ID,Create Date,Customer Name,Campaign,Nickname,WORKING STATUS
25556,107558,2021-11-24 20:52:47.514,HELENA CHEMICAL COMPANY,REACTIVE,US Commercial Missed Attach Mari 20211130,DECLINED
25557,107551,2021-11-24 20:52:11.648,BAPTIST MEMORIAL HEALTH CARE CORPORATION,REACTIVE,US Commercial Missed Attach Mari 20211130,DECLINED
25558,107545,2021-11-24 20:51:41.654,VALEO,REACTIVE,US Commercial Missed Attach Mari 20211130,READOUT DONE
25559,107464,2021-11-24 20:44:41.116,SPECTRUM BRANDS,REACTIVE,US Commercial Missed Attach Mari 20211130,READOUT DONE
25560,107427,2021-11-24 20:41:00.480,MUSEUM OF SCIENCE AND INDUSTRY,REACTIVE,US Commercial Missed Attach Mari 20211130,READOUT DONE
...,...,...,...,...,...,...
36402,107775,2021-11-24 21:13:15.642,PURE STORAGE,REACTIVE,US Commercial Missed Attach Mari 20211130,READOUT DONE
36403,107773,2021-11-24 21:13:03.574,ROSS STORES,REACTIVE,US Commercial Missed Attach Mari 20211130,READOUT DONE
36404,107774,2021-11-24 21:13:08.899,MADERA COMMUNITY HOSPITAL INC,REACTIVE,US Commercial Missed Attach Mari 20211130,READOUT DONE
36533,107904,2021-11-24 21:34:06.418,PARKVIEW HEALTH SYSTEMS,REACTIVE,US Commercial Missed Attach Mari 20211130,DECLINED


In [26]:
# booked_requestids = nonbooked.loc[nonbooked['TARGET_REQUEST_ID'].isin(finbi_requests),['Request ID','Forecast Stage', 'Customer Name','Deal ID','Compass Campaign Name']]

In [27]:
# booked_requestids.sort_values(by=['Sheet Name','Request ID'], inplace=True)

In [28]:
# booked_requestids.shape[0]

In [29]:
# booked_requestids.loc[booked_requestids['Forecast Stage']!='5 - Closed Won'].to_excel(r'C:\Users\phsheari\Documents\FIN BI Data\REQUEST IDS TO BOOK.xlsx', index=False)
# booked_requestids.loc[booked_requestids['Forecast Stage']!='5 - Closed Won'].to_pickle(r'C:\Users\phsheari\Documents\FIN BI Data\REQUEST IDS TO BOOK.pkl')

### Begin to Inspect, Clean, & Curate the remaining dataframe

In [30]:
nonbooked['DELIVERABILITY'].value_counts()

Good                 35954
Insufficient Data     6763
Name: DELIVERABILITY, dtype: int64

In [31]:
hyper_csv_file_path_tracker = r'C:\Users\phsheari\Documents\Compass_hyper_csvs\compass_tracker_hyper.csv'
excel_file_path_tracker = r'C:\Users\phsheari\Documents\Compass Trackers\ConsolidatedReport_' + today_file + '.xlsx'
excel_file_path_sfdc_version = r'C:\Users\phsheari\Documents\SFDC Data\PIPELINE UPDATE\SFDC_Trackerfile.pkl'
excel_file_path_tableau = r'C:\Users\phsheari\Documents\Compass Tableau Data\ConsolidatedReport_' + today_file + '.xlsx'

In [32]:
# nonbooked = nonbooked.reindex(columns = booked.columns)
# nonbooked = nonbooked.loc[~(nonbooked['Forecast Stage']=='5 - Closed Won')]

In [33]:
#Determine what is undeliverable so you can avoid or remove any pipeline values in those records

undeliverable_requests = list(nonbooked.loc[(nonbooked['INSUFFICIENT_DATA_COUNT']==1)]['Request ID'])

nonbooked.loc[nonbooked['Request ID'].isin(undeliverable_requests),'Forecast Stage']=pd.NA
#nonbooked.loc[nonbooked['Request ID'].isin(undeliverable_requests),'Forecast Status']=pd.NA
#nonbooked.loc[nonbooked['Request ID'].isin(undeliverable_requests),'Opportunity Name']=pd.NA
#nonbooked.loc[nonbooked['Request ID'].isin(undeliverable_requests),'Projected Booking ($,000)']=pd.NA
#nonbooked.loc[nonbooked['Request ID'].isin(undeliverable_requests),'Target Fiscal Month']=pd.NA
#nonbooked.loc[nonbooked['Request ID'].isin(undeliverable_requests),'Target Fiscal Quarter']=pd.NA
#nonbooked.loc[nonbooked['Request ID'].isin(undeliverable_requests),'Target Fiscal Year']=pd.NA

In [34]:
def update_did_dupechk(did, dupechk):
    if pd.isnull(did):
        return False
    elif did == None:
        return False
    elif did in [-44444,99999,9999,22222]:
        return False
    else:
        return dupechk

In [35]:
# Initialize the nonbooked dataframe Dupechk field to 0. Sort the Deal ID Subset dataframe to determine which rows contain duplicate deal IDs
nonbooked['Dupechk'] = 0

did_subset = nonbooked.loc[(nonbooked['INSUFFICIENT_DATA_COUNT']==0),['Request ID','Create Date','Customer Name', 'Deal ID','Forecast Stage']].copy()
did_subset.sort_values(['Deal ID','Request ID','Create Date'], ascending=[True, False, False], ignore_index=False, inplace=True)
did_subset['Dupechk'] = did_subset.duplicated(['Deal ID'], keep='first')  # Chronologically sets the first deal id occurance to False, all after to True
did_subset['Dupechk'] = did_subset.apply(lambda x: update_did_dupechk(x['Deal ID'], x['Dupechk']), axis=1)

In [36]:
dupes_requestid_list = list(did_subset.loc[(did_subset['Dupechk']==True) & ~(did_subset['Forecast Stage']=='5 - Closed Won')]['Request ID'])

In [37]:
len(dupes_requestid_list)

3683

In [38]:
# did_subset.to_excel(r'C:\Users\phsheari\Desktop\Dupes.xlsx', index=False)

In [39]:
# nonbooked.info()

In [40]:
# booked.info()

In [41]:
# nonbooked.loc[nonbooked['Request ID'].isin(dupes_requestid_list)]   ### Sanity Check

In [42]:
# Set the duplicate deal ID values so that pipeline values are not duplicated
nonbooked.loc[nonbooked['Request ID'].isin(dupes_requestid_list),'Dupechk'] = 1
nonbooked.loc[nonbooked['Request ID'].isin(dupes_requestid_list),'Forecast Stage'] = '7 - Duplicate Request'
#nonbooked.loc[nonbooked['Request ID'].isin(dupes_requestid_list),'Forecast Status'] = pd.NA
#nonbooked.loc[nonbooked['Request ID'].isin(dupes_requestid_list),'Opportunity Name'] = pd.NA
#nonbooked.loc[nonbooked['Request ID'].isin(dupes_requestid_list),'Projected Booking ($,000)'] = pd.NA
#nonbooked.loc[nonbooked['Request ID'].isin(dupes_requestid_list),'Target Fiscal Month'] = pd.NA
#nonbooked.loc[nonbooked['Request ID'].isin(dupes_requestid_list),'Target Fiscal Quarter'] = pd.NA
#nonbooked.loc[nonbooked['Request ID'].isin(dupes_requestid_list),'Target Fiscal Year'] = pd.NA

In [43]:
#df = pd.concat([booked, nonbooked]) #### [booked, nonbooked]
df = nonbooked

In [44]:
df.shape

(42717, 110)

### Change the field name "BDM Assigned" to "Compass Advisor"

In [45]:
df.rename(columns={'BDM Assigned': 'Compass Advisor'}, inplace=True)

### Read in the Compass Advisor Email Lookup Table to resolve proper names

In [46]:
advisor_emails = pd.read_excel(r'C:\Users\phsheari\Documents\Compass Advisors\Advisor Email Translation.xlsx', usecols=['cecid','name', 'source'])
offer_creator_emails = pd.read_excel(r'C:\Users\phsheari\Documents\Compass LPM_Offer_Creators\Offer Creator team list 3-1-2022.xlsx', usecols=['email','name', 'vendor'])

In [47]:
advisor_email = dict(zip(advisor_emails.cecid, advisor_emails.name))
offer_email = dict(zip(offer_creator_emails.email, offer_creator_emails.name))

In [48]:
offer_email

{'andreshe@cisco.com': 'Andres Mauricio Hernandez ',
 'anosilva@cisco.com': 'Antonio Silva',
 'ansaenz@cisco.com': 'Andres Saenz',
 'apuertoq@cisco.com': 'Andres Felipe Puerto',
 'borirodr@cisco.com': 'Boris Rodriguez',
 'camdiaz@cisco.com': 'Camilo Diaz',
 'carguill@cisco.com': 'Carolina Guillermo',
 'chrmedin@cisco.com': 'Christian Medina',
 'daniels3@cisco.com': 'Daniel Glennie',
 'davvasqu@cisco.com': 'David Alejandro Vasquez Vargas',
 'ddiazriv@cisco.com': 'Daniel Steven Diaz Rivera',
 'ecuello@cisco.com': 'Edgar Felipe Cuello Betancourt',
 'edprieto@cisco.com': 'Alexander Prieto',
 'emaximo@cisco.com': 'Ella Maximo',
 'ggomezbe@cisco.com': 'Gladys Adriana Gomez Beltran',
 'gileon@cisco.com': 'Giovanni Leon',
 'jmateus@cisco.com': 'Jeferson Mateus',
 'jmorenoa@cisco.com': 'Sebastian Moreno',
 'jnorenag@cisco.com': 'Julian Norena',
 'jofiguei@cisco.com': 'Joao Figueiredo',
 'mpalheir@cisco.com': 'Miguel Palheira Pinto',
 'mvieiraf@cisco.com': 'Lourdes Fortes',
 'nicolaro@cisco.com'

In [49]:
advisor_vendor = dict(zip(advisor_emails.cecid, advisor_emails.source))
oc_vendor = dict(zip(offer_creator_emails.email, offer_creator_emails.vendor))

In [50]:
def get_advisor_name_from_cecid(advisor):
    if pd.isnull(advisor) or advisor == None or advisor.lower()=='nan':
        return None
    if advisor=='Expired' or advisor=='N/A':
        return 'Expired'
    if advisor.lower().endswith('cisco.com'):
        pieces = advisor.lower().split('@')
        cecid = pieces[0]
        actual_name = advisor_email.get(cecid,'Term')
        return actual_name 
    else:
        return advisor

In [51]:
def get_advisor_vendor_from_cecid(advisor):
    if pd.isnull(advisor) or advisor == None or advisor.lower()=='nan':
        return None
    if advisor=='Expired' or advisor=='N/A':
        return None
    if advisor.lower().endswith('cisco.com'):
        pieces = advisor.lower().split('@')
        cecid = pieces[0]
        vendor_name = advisor_vendor.get(cecid, None)
        if vendor_name:
            return vendor_name.upper()
        else:
            return None
    else:
        return None

In [52]:
df['NewAdvisor'] = df['Compass Advisor'].apply(lambda x: get_advisor_name_from_cecid(x))

In [53]:
df['Vendor'] = df['Compass Advisor'].apply(lambda x: get_advisor_vendor_from_cecid(x))

In [54]:
df['Concierge Advisor Name'] = df['CONCIERGE_ADVISOR_ASSIGNED'].apply(lambda x: get_advisor_name_from_cecid(x))

In [55]:
df['Concierge Advisor Name'].unique()

array([None, 'Mark Vodka', 'Peter Wittenstrom', 'Emma Baird',
       'Alicia Biersteker', 'Martin Coup', 'Brenda Buck',
       'Stacey Edwards', 'Jenee St John', 'Tracy Turner',
       'Bill Black-Hogins', 'Jenee St. John', 'Alicia Bierstkeker',
       'Stacey Edwards,', 'Atif Ahmad', 'Clare Fagan',
       'DUPLICATE of SAME record', 'Adrian Machado',
       'Jenee St John <jestjohn@cisco.com>', 'x', 'Term',
       'Alicia Bierstker', 'Alicia', 'Houman Asefi', 'Sangeetha Sampath',
       'Gloria Canales', 'Jayjay Towler'], dtype=object)

In [56]:
df['Compass Advisor'] = df['NewAdvisor']

In [57]:
# df['Compass Advisor'].unique()

In [58]:
df.drop(columns=['NewAdvisor'], inplace=True)

In [59]:
df['Vendor'].unique()

array(['MODIS INC', 'NUB78', 'ADECCO UK LTD', 'SERVICESOURCE', None,
       'ADECCO AISAPAC', 'CISCO', 'ADECCO AUSTRALIA'], dtype=object)

In [60]:
# column_list = df.columns
# [col for col in column_list if 'LPM' in col]

In [61]:
def get_oc_name_from_email(offer_creator_email):
    if pd.isnull(offer_creator_email) or offer_creator_email == None or offer_creator_email.lower()=='nan':
        return None

    if offer_creator_email.lower().endswith('cisco.com'):
        actual_name = offer_email.get(offer_creator_email,pd.NA)
        return actual_name 
    else:
        return offer_creator_email

In [62]:
# df.loc[df['LPM_EMAIL'].notnull()]['LPM_EMAIL']

In [63]:
df['Offer_Creator_Name'] = df['OFFER_CREATOR_EMAIL'].apply(lambda x: get_oc_name_from_email(x))

In [64]:
df.loc[df['Offer_Creator_Name'].notnull()][['Offer_Creator_Name']]

,Offer_Creator_Name
18,Rafael Ernesto Porras Diaz
19,Rafael Ernesto Porras Diaz
20,Rafael Ernesto Porras Diaz
21,Rafael Ernesto Porras Diaz
22,Rafael Ernesto Porras Diaz
...,...
42711,Gladys Adriana Gomez Beltran
42712,Gladys Adriana Gomez Beltran
42713,Gladys Adriana Gomez Beltran
42714,Gladys Adriana Gomez Beltran


In [65]:
def get_oc_vendor_from_email(offer_creator_email):
    if pd.isnull(offer_creator_email) or offer_creator_email == None or offer_creator_email.lower()=='nan':
        return None

    if offer_creator_email.lower().endswith('cisco.com'):
        vendor = oc_vendor.get(offer_creator_email,pd.NA)
        return vendor 
    else:
        return offer_creator_email

In [66]:
df['Offer_Creator_Vendor'] = df['OFFER_CREATOR_EMAIL'].apply(lambda x: get_oc_vendor_from_email(x))

### Fix/Clean Customer Names

In [67]:
name_correction = {'201820184589':'NIPPON TELEGRAPH AND TELEPHONE EAST CORPORATION'}

In [68]:
def correct_customer_name(df):
    reqid = df['TARGET_REQUEST_ID']
    if reqid in name_correction.keys():
        return name_correction.get(reqid)
    else:
        return df['Customer Name'].strip()

In [69]:
# df['Customer Name'] = df['Customer Name'].astype(str)
# df['Customer Name'] = df.apply(lambda x: correct_customer_name(x), axis=1)

In [70]:
df['Customer Name'] = df['Customer Name'].str.lower().str.replace('\^_ inc',' inc').str.replace('\^_ ',', ').str.replace(', ',' ').str.replace(' inc.',' inc ').str.replace('\* ','').str.strip().str.upper()

<ipython-input-70-c82f039ad48b>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Customer Name'] = df['Customer Name'].str.lower().str.replace('\^_ inc',' inc').str.replace('\^_ ',', ').str.replace(', ',' ').str.replace(' inc.',' inc ').str.replace('\* ','').str.strip().str.upper()


In [71]:
def squeeze_customer_name(customer_name):
    if type(customer_name)==float:
        return ''
    if customer_name.startswith('THE '):
        customer_name = customer_name[4:]
    customer_name = customer_name.replace(',',"").replace('[ACTIVE]','').replace('-','').replace('_','').replace('.','').replace(' ','').strip()
    return customer_name

In [72]:
df['Squeezed'] = df['Customer Name'].apply(lambda x: squeeze_customer_name(x))

In [73]:
df['Squeezed']

0        ADOBESYSTEMSINCRPORATED
1                   DILLARD'SINC
2        NEIMANMARCUSGROUPINCTHE
3        ARCTICSLOPEREGIONALCORP
4                    ILLUMINAINC
                  ...           
42712            TROYCORPORATION
42713                     VALPAK
42714              UNITEDRENTALS
42715                  VERSARINC
42716                  VTSYSTEMS
Name: Squeezed, Length: 42717, dtype: object

In [74]:
df['Prepend'] = df['Squeezed'].apply(lambda x: x[:2].upper())

In [75]:
# df.loc[(df['Customer Name'].str.contains('choctaw', case=False))] #df['GUID'].isnull()) | 

### Standardize Campaign Name to Uppercase

In [76]:
df['Campaign'] = df['Campaign'].str.upper()

In [77]:
df.loc[df['Campaign'].str.contains('missed attach', na=False, case=False),'Campaign'] = df.loc[df['Campaign'].str.contains('missed attach', na=False, case=False), 'Campaign'].apply(lambda x: x.replace('MISSED ATTACH','COVERAGE PROGRAM'))

In [78]:
df.loc[df['Campaign'].str.contains('coverage p', na=False, case=False)][['Request ID','Customer Name','Campaign']]

,Request ID,Customer Name,Campaign
25608,103167,PARTNERS HEALTHCARE SYSTEM,COVERAGE PROGRAM - SNTC PROACTIVE ATTACH
25609,103160,HEALTH CARE SERVICES CORP,COVERAGE PROGRAM - SNTC PROACTIVE ATTACH
25610,105574,UNITED NATIONS_UNITED NATIONS,COVERAGE PROGRAM - TELEMETRY
25611,105477,POLSKIE KOLEJE PANSTWOWE,COVERAGE PROGRAM - TELEMETRY
25612,105345,LAUREATE CRI,COVERAGE PROGRAM - TELEMETRY
...,...,...,...
42712,114327,TROY CORPORATION,COVERAGE PROGRAM - TELEMETRY
42713,114365,VALPAK,COVERAGE PROGRAM - TELEMETRY
42714,114344,UNITED RENTALS,COVERAGE PROGRAM - TELEMETRY
42715,114370,VERSAR INC,COVERAGE PROGRAM - TELEMETRY


In [79]:
# df.loc[df['Requester Name'].str.lower().isin(confirm_exclusions)][['Request ID','Requester Name']]

### Clean up issues with Requester Name

In [80]:
confirm_exclusions = ['kenpatton@cisco.com','kpatton@cisco.com','lsweidan@cisco.com','escudder@cisco.com', 'rosecam@cisco.com']

In [81]:
def swap_requester_name_with_notified(requester_name, distribution_list, campaign_name):
    if requester_name:
        requester_name = str(requester_name.lower())
    if distribution_list:
        distribution_list = str(distribution_list)
    if (campaign_name != 'nan' or pd.notnull(campaign_name)):
        campaign_name = str(campaign_name).lower()
     
    if ('target' in campaign_name or 'tac lead' in campaign_name or 'success tracks' in campaign_name or 'major' in campaign_name) and (requester_name.lower() in confirm_exclusions and pd.notnull(distribution_list)):
        requester_name = distribution_list
        return requester_name.upper()
    else:
        return requester_name.upper()
    

In [82]:
df.loc[df['Requester Name'].notnull(),'Requester Name'] = df.loc[df['Requester Name'].notnull()].apply(lambda x: swap_requester_name_with_notified(x['Requester Name'],x['DISTRIBUTION_LIST'], x['Campaign']), axis=1)

### Sanity Check

In [83]:
# df.loc[(df['Compass Campaign Name']=='BCS 3.0 TARGETING') & (df['Who Should be Notified On Completion of Analysis'].notnull())][['Request ID','Customer Name','Requester Name','Who Should be Notified On Completion of Analysis']]

In [84]:
# name_counts = pd.DataFrame(df['Requester Name'].value_counts())

In [85]:
# name_counts.reset_index(inplace=True)

In [86]:
# name_counts

In [87]:
# df.loc[df['Requester Name']=='nan']

### Set Recipient Confirmation Status & Clean Requester Name

In [88]:
def set_recipient_confirmation_field(requester_name, compass_url, oa_url, distribution_list):
    #requester_name = str(requester_name)

    if pd.isnull(requester_name) or requester_name == 'N/A' or requester_name == 'nan' or type(requester_name) == bool or requester_name.lower().startswith('kpatton'):
        return "NOT CONFIRMED"
    
    elif (pd.isnull(compass_url) and pd.isnull(oa_url)):
          return 'NOT CONFIRMED'
    
    elif requester_name.lower() in confirm_exclusions and not pd.isnull(distribution_list):
        return "CONFIRMED"
    
    elif requester_name.lower() in confirm_exclusions  and pd.isnull(distribution_list):
        return "NOT CONFIRMED"
    
    else:
        return "CONFIRMED"

In [89]:
df['Recipient Confirmation'] = df.apply(lambda x: set_recipient_confirmation_field(x['Requester Name'], x['COMPASS URL'], x['OA URL'], x['DISTRIBUTION_LIST']), axis=1)

In [90]:
df['Recipient Confirmation'].value_counts()

CONFIRMED        28861
NOT CONFIRMED    13856
Name: Recipient Confirmation, dtype: int64

In [91]:
working_status_list = ['ASSIGNED','BOOKED','EXCEPTION_REVIEW','READOUT DONE','READOUT POSTPONED','SCHEDULED','UNASSIGNED']

In [92]:
# df.loc[(df['Request FY']=='2022') & (df['Report FY']=='2022') & (df['WORKING STATUS'].isin(working_status_list)) & (df['Recipient Confirmation']=='NOT CONFIRMED')][['Requester Name','DISTRIBUTION_LIST','Recipient Confirmation']]

### Note: The Requester Name is acting as if there is hidden / blank / characters. Test deeply!!!

In [93]:
# requester_list = list(df['Requester Name'])

In [94]:
# len(requester_list)

In [95]:
# confirm_list = list([set_recipient_confirmation_field(x) for x in requester_list])

In [96]:
# len(confirm_list)

In [97]:
# kk = pd.DataFrame(confirm_list)

In [98]:
# kk.rename(columns={0:'Recipient Confirmation'}, inplace=True)

In [99]:
# kk.value_counts()

In [100]:
df.loc[df['Recipient Confirmation'].isin(['NOT CONFIRMED'])][['Request ID','Create Date','Customer Name','GUID','Deal ID','Requester Name','Campaign']].shape

(13856, 7)

In [101]:
# def fix_requester_name(requester_name, sss_confirmation):
#     requester_name = str(requester_name)
#     sss_confirmation = str(sss_confirmation)
    
#     if sss_confirmation.lower() == "not confirmed":
#         return requester_name
    
#     if pd.isnull(requester_name) or requester_name == 'nan':
#         return pd.NA
    
#     if requester_name.lower().endswith("@cisco.com"):
#             return requester_name
#     else:    # requester name does not end in "@cisco.com"
#         if not ',' in requester_name:   # would not be construed as a list of items
#                 return requester_name + "@cisco.com"
#         else:
#             requester_name = requester_name.replace(';',',')
#             alist = list(requester_name.split(','))
#             checklist = []
#             for n in alist:
#                 checklist.append(n + "@cisco.com")
#             return ', '.join(checklist)

In [102]:
# df['Requester Name'] = df.apply(lambda x: fix_requester_name(x['Requester Name'],x['Recipient Confirmation']), axis = 1)

In [103]:
# df['Requester Name'].value_counts()

In [104]:
# df['Requester Name'] = df['Requester Name'].apply(lambda x: None if x == 'nan@cisco.com' else x)

### Assign Compass Advisor name

In [105]:
def clean_bdm_assigned(bdm_assigned):
    if pd.isnull(bdm_assigned):
        return None
    elif bdm_assigned.lower() == 'nan':
        return 'Expired'
    else:
        return bdm_assigned.title()

In [106]:
df['Compass Advisor'] = df['Compass Advisor'].apply(clean_bdm_assigned)

In [107]:
# df.loc[df['Recipient Confirmation'] == 'Confirmed',['Request ID','Requester Name','Compass Campaign Name','Recipient Confirmation']].shape

In [108]:
# df['Compass Advisor'].unique()

In [109]:
confirmed = df.loc[(df['Recipient Confirmation']=='Confirmed') & ~(df['Requester Name'].str.lower().isin(confirm_exclusions)) & (df['Requester Name'].notnull())][['Request ID', 'Create Date','Customer Name','Requester Name','Campaign','Sales Level 2','Sales Level 3','Recipient Confirmation', 'Compass Advisor','Readout_Date']]

In [110]:
# Sanity Check -- Test if we show all the "Confirmed" recipients for readouts on Targeting Campaign leads
confirmed

,Request ID,Create Date,Customer Name,Requester Name,Campaign,Sales Level 2,Sales Level 3,Recipient Confirmation,Compass Advisor,Readout_Date


In [111]:
def populate_sl3_from_requestername(requester_name):
    if requester_name in sssgeo_dict.keys():
        try:
            return sssgeo_dict.get(requester_name)
        except:
            return 'No SL3 Available'

confirmed['Sales Level 3'] = confirmed.apply(lambda x: populate_sl3_from_requestername(x['Requester Name']), axis=1)

In [112]:
confirmed = confirmed[['Request ID', 'Create Date', 'Customer Name', 'Requester Name','Recipient Confirmation','Campaign', 'Sales Level 2', 'Sales Level 3',]]

### Fix the Sales Level3 value for certain Requester Names

In [113]:
# df.loc[df['Request ID'].isin(list(confirmed['Request ID'])),'Lvl3'] = df.loc[df['Request ID'].isin(list(confirmed['Request ID']))].apply(lambda x: populate_sl3_from_requestername(x['Requester Name']), axis=1)

### Standardize the Partner Names from the partner_dict hash

In [114]:
PartnerNames = list(df.loc[df['Partner Name'].notnull()]['Partner Name'].unique())

In [115]:
PartnerNames.sort()

In [116]:
# PartnerNames

In [117]:
# partner_dict
def update_partner_names(partner_name):
    if partner_name == np.nan or partner_name is None or pd.isnull(partner_name):
        return None
    else:
        PN = partner_name.lower().replace('. ','').replace('.','')
        newname = partner_dict.get(PN, PN.upper())
        return newname

In [118]:
df['Partner Name'] = df['Partner Name'].apply(update_partner_names)

### Apply Fiscal Period Dating to the Create Date and Readout Date fields

In [119]:
### Fix Injection & Readout Date for 10 Missed Attach records
coverage_at_renewal_q1_list = ['104315','104316','104317','104318','104319','104320','104321','104322','104323','104324']    #['104315','104316','104317','104318','104319','104320','104321','104322','104323','104324']    #[104315,104316,104317,104318,104319,104320,104321,104322,104323,104324]   

In [120]:
df.loc[df['Request ID'].isin(coverage_at_renewal_q1_list), 'Create Date'] = pd.to_datetime('20211101T1903', format='%Y-%m-%d %H:%M:%S', errors='ignore')

In [121]:
df.loc[df['Request ID'].isin(coverage_at_renewal_q1_list), 'Readout_Date'] = pd.to_datetime('20211103', format='%Y-%m-%d', errors='ignore')

In [122]:
df.loc[df['Request ID'].isin(coverage_at_renewal_q1_list) ][['Request ID','Create Date','Readout_Date']]

,Request ID,Create Date,Readout_Date
28298,104315,2021-11-01 19:03:00,2021-11-03
28299,104316,2021-11-01 19:03:00,2021-11-03
28300,104317,2021-11-01 19:03:00,2021-11-03
28301,104324,2021-11-01 19:03:00,2021-11-03
28302,104318,2021-11-01 19:03:00,2021-11-03
28303,104319,2021-11-01 19:03:00,2021-11-03
28304,104320,2021-11-01 19:03:00,2021-11-03
28305,104323,2021-11-01 19:03:00,2021-11-03
28306,104321,2021-11-01 19:03:00,2021-11-03
28307,104322,2021-11-01 19:03:00,2021-11-03


In [123]:
df.loc[df['Campaign'].str.contains('coverage at', na=False, case=False) & (df['Readout_Date'] < '2021-10-31')]

,Request ID,Create Date,Created By,Request Type,Modified,Modified By,Date Completed,LAST_UPDATED,QA Approve,REVIEW_REQUIRED_FLAG,...,Nickname,Forecast Stage,Dupechk,Vendor,Concierge Advisor Name,Offer_Creator_Name,Offer_Creator_Vendor,Squeezed,Prepend,Recipient Confirmation


In [124]:
def get_fiscal_half(fiscal_quarter):
    if pd.isnull(fiscal_quarter):
        return pd.NA
    elif fiscal_quarter == '1' or fiscal_quarter == '2':
        return '1'
    else:
        return '2'

In [125]:
df['Request FY'] = df['Create Date'].apply(fc.get_fiscal_year)

In [126]:
df['Request FQ'] = df['Create Date'].apply(fc.get_fiscal_quarter)

In [127]:
df['Request FM'] = df['Create Date'].apply(fc.get_fiscal_month)

In [128]:
df['Request FWOFM'] = df['Create Date'].apply(fc.get_fiscal_week_of_fiscal_month)

In [129]:
df['Request FQ'].unique()

array(['3', '4', '1', '2'], dtype=object)

In [130]:
df['Request FH'] = df['Request FQ'].apply(get_fiscal_half)

In [131]:
df['Request FWOFQ'] = df['Create Date'].apply(fc.get_fiscal_week_of_fiscal_quarter)

In [132]:
df['Request FWOFY'] = df['Create Date'].apply(fc.get_fiscal_week_of_fiscal_year)

In [133]:
df['Readout_Date'] = pd.to_datetime(df['Readout_Date'])

In [134]:
df['Readout FY'] = df['Readout_Date'].apply(fc.get_fiscal_year)
df['Readout FQ'] = df['Readout_Date'].apply(fc.get_fiscal_quarter)
df['Readout FM'] = df['Readout_Date'].apply(fc.get_fiscal_month)

In [135]:
df['Readout FWOFM'] = df['Readout_Date'].apply(fc.get_fiscal_week_of_fiscal_month)

In [136]:
df['Readout FWOFQ'] = df['Readout_Date'].apply(fc.get_fiscal_week_of_fiscal_quarter)

In [137]:
df['Readout FWOFY'] = df['Readout_Date'].apply(fc.get_fiscal_week_of_fiscal_year)

In [138]:
df['Readout FH'] = df['Readout FQ'].apply(get_fiscal_half)

In [139]:
df['Request FH'].unique()

array(['2', '1'], dtype=object)

In [140]:
df['OC Readout FY'] = df['OFFER_CREATOR_READOUT_DT'].apply(fc.get_fiscal_year)

In [141]:
df['OC Readout FQ'] = df['OFFER_CREATOR_READOUT_DT'].apply(fc.get_fiscal_quarter)

In [142]:
df['OC Readout FM'] = df['OFFER_CREATOR_READOUT_DT'].apply(fc.get_fiscal_month)

### Set the Campaign Type field from the campaign names

In [143]:
def set_campaign_type(campaign_name):
    if pd.isnull(campaign_name) or campaign_name == '':
        return 'NO CAMPAIGN'
    
    elif campaign_name.lower() == 'test' or campaign_name.lower() == 'accelerator' or campaign_name.lower() == 'proactive - ignore' \
    or campaign_name.upper() == 'IAP – STEPHAN' or campaign_name.upper() == 'IAP - STEPHAN' or 'research' in campaign_name.lower():
        return 'NONREPORTING/TESTS'
    
    elif 'cxib scale' in campaign_name.lower():
        return 'COVERAGE AT RENEWAL'
    
    elif 'cxib' in campaign_name.lower():
        return 'CXIB'    
    
    elif 'coverage at renewal' in campaign_name.lower():
        return 'COVERAGE AT RENEWAL'
    
    elif campaign_name.upper() == 'COVERAGE PROGRAM - TELEMETRY':
        return 'COVERAGE PROGRAM'
    
    elif ('missed attach' in campaign_name.lower() or 'coverage program' in campaign_name.lower()) and ('sntc tac leads' in campaign_name.lower() or 'tac sntc' in campaign_name.lower() or campaign_name.upper()=='PROACTIVE - TAC LEAD'):
        return 'COVERAGE PROGRAM'
    
    elif ('missed attach' in campaign_name.lower() or 'coverage program' in campaign_name.lower()) and ('proactive attach' in campaign_name.lower() or 'proactive sntc' in campaign_name.lower()):
        return 'COVERAGE PROGRAM'
    
    elif ('missed attach' in campaign_name.lower() or 'coverage program' in campaign_name.lower()) and ('sspt' in campaign_name.lower() or 'telemetry' in campaign_name.lower()):
        return 'COVERAGE PROGRAM'
    
    elif ('missed attach' in campaign_name.lower() or 'coverage program' in campaign_name.lower()) and ('field' in campaign_name.lower() or 'reactive' in campaign_name.lower()):
        return 'FIELD'
    
    #elif ('missed attach' in campaign_name.lower() or 'coverage program' in campaign_name.lower()):
    #    return 'PROACTIVE COVERAGE'
    
    elif 'proactive - refresh bdm' in campaign_name.lower():   #'PROACTIVE - AMERICAS - IRR EXPAND',
        return 'NONREPORTING/TESTS'
        
    elif 'proactive' in campaign_name.lower() or campaign_name.upper() in ('PROACTIVE - REFRESH','PROACTIVE - CS RESEARCH'):   #'PROACTIVE - AMERICAS - IRR EXPAND',
        return 'PROACTIVE_OLD'
    
    elif 'proactive tac' in campaign_name.lower():
        return 'COVERAGE PROGRAM'
    
    elif 'reactive' in campaign_name.lower() or campaign_name.upper() in ('UKI - SUZI','UKI -- SUZI','PROACTIVE - PARTNER.1','PROACTIVE - PARTNER.2') or campaign_name.upper() == 'PROACTIVE - REFRESH BDM':   #'PROACTIVE - AMERICAS - IRR EXPAND',
        return 'FIELD'
    
    elif campaign_name.upper() == 'TELEMETRY':
        return 'TELEMETRY'
    
    elif 'bcs' in campaign_name.lower() or 'success' in campaign_name.lower() or 'solution' in campaign_name.lower() or 'sspt' in campaign_name.lower() or 'st target' in campaign_name.lower() or 'ctu' in campaign_name.lower() or 'target' in campaign_name.lower():
        return 'TARGETING'    

In [144]:
df['Campaign Type'] = df['Campaign'].apply(lambda x: set_campaign_type(x))

In [145]:
# df['Campaign Type'].unique()
df.loc[df['Campaign'].str.contains('CXIB', na=False, case=False)]['Campaign Type'].unique()

array(['CXIB'], dtype=object)

In [146]:
df.loc[df['Campaign Type'].isnull()]['Campaign'].unique()

array(['TEST REQUEST'], dtype=object)

### Set the Items Recommended fields

In [147]:
def bcs_recommended(items_recommended):
    if pd.isnull(items_recommended):
        return None
    elif 'BCS' in items_recommended:
        return 'BCS'
    else:
        return None

In [148]:
# df['Items Recommended'].unique()

In [149]:
df['IR_BCS'] = df['Items Recommended'].apply(bcs_recommended)

In [150]:
def ec_recommended(items_recommended):
    if pd.isnull(items_recommended):
        return None
    elif 'EC-' in items_recommended:
        return 'EC'
    else:
        return None

In [151]:
df['IR_EC'] = df['Items Recommended'].apply(ec_recommended)

In [152]:
def dnac_recommended(items_recommended):
    if pd.isnull(items_recommended):
        return None
    elif 'DNA' in items_recommended:
        return 'DNAC'
    else:
        return None

In [153]:
df['IR_DNAC'] = df['Items Recommended'].apply(dnac_recommended)

In [154]:
def sspt_recommended(items_recommended):
    if pd.isnull(items_recommended):
        return None
    elif 'SSPT' in items_recommended:
        return 'SSPT'
    else:
        return None

In [155]:
df['IR_SSPT'] = df['Items Recommended'].apply(sspt_recommended)

In [156]:
def st_recommended(items_recommended):
    if pd.isnull(items_recommended):
        return None
    elif 'ST' in items_recommended:
        return 'ST'
    else:
        return None

In [157]:
df['IR_ST'] = df['Items Recommended'].apply(st_recommended)

In [158]:
def sntc_recommended(items_recommended):
    if pd.isnull(items_recommended):
        return None
    elif 'SNTC' in items_recommended:
        return 'SNTC'
    else:
        return None

In [159]:
df['IR_SNTC'] = df['Items Recommended'].apply(sntc_recommended)

In [160]:
def irr_recommended(items_recommended):
    if pd.isnull(items_recommended):
        return None
    elif 'IRR' in items_recommended:
        return 'IRR'
    else:
        return None

In [161]:
df['IR_IRR'] = df['Items Recommended'].apply(irr_recommended)

In [162]:
def productrefresh_recommended(items_recommended):
    if pd.isnull(items_recommended):
        return None
    elif 'PR' in items_recommended:
        return 'PR'
    else:
        return None

In [163]:
df['IR_PR'] = df['Items Recommended'].apply(productrefresh_recommended)

In [164]:
def brw_recommended(items_recommended):
    if pd.isnull(items_recommended):
        return None
    elif 'BRW' in items_recommended:
        return 'BRW'
    else:
        return None

In [165]:
df['IR_BRW'] = df['Items Recommended'].apply(brw_recommended)

# Setup Seller Confirmation conditions

In [166]:
requesters_to_remove_2021 = list(['alcastri@cisco.cm@cisco.com','alcastri@cisco.com','anamoren@cisco.com','antmills@cisco.com','avillalo@cisco.com','boenglis@cisco.com',
                                 'borirodr@cisco.com','braolive@cisco.com','brbuck@cisco.com','camruiz@cisco.com','fpizano@cisco.com', 'jbaldera@cisco.com','jcollie2@cisco.com',
                                 'josguti2@cisco.com','jszmak@cisco.com','kpatton@cisco.com','kywatts@cisco.com','lmaldon@cisco.com', 'lsweidan@cisco.com' , 'nilosada@cisco.com',
                                 'nnino@cisco.com','qhopkins@cisco.com','stotero@cisco.com', 'wzeliger@cisco.com','caltree@cisco.com','obustosm@cisco.com','antchris@cisco.com',
                                 'dadiaspe@cisco.com','jforgy@cisco.com','jguggehe@cisco.com','stehartm@cisco.com','abdulahm@cisco.com','pamohan@cisco.com','charpost@cisco.com',
                                 'mragam@cisco.com','javalenc@cisco.com','escudder@cisco.com'])

In [167]:
campaigns_to_remove_from_count_2021 = list(['ACCELERATOR','IAP - STEPHAN','PROACTIVE - AMERICAS - F20Q2-3 RENEW (140)','PROACTIVE - AMERICAS - IRR EXPAND', 'PROACTIVE - AMERICAS - US COMM.1','PROACTIVE - AMERICAS - US COM.1',
                                       'PROACTIVE - BCS 3.0', 'PROACTIVE - CAP.1','PROACTIVE - CS RESEARCH', 'PROACTIVE - EMEAR.BETA','PROACTIVE - IGNORE','PROACTIVE - PARTNER.1','PROACTIVE - PARTNER.2', 'PROACTIVE - PRESALES ALIGN PITSTOP EFFORT',
                                       'PROACTIVE - REFRESH LIST', 'PROACTIVE - SW CONFORMANCE', 'PROACTIVE - SW CONFORMANCE.2', 'PROACTIVE - TAC LEAD', 'PROACTIVE - USPS.1','REACTIVE - APJC GC ENT', 'REACTIVE - APJC.1','REACTIVE - APJC.BETA',
                                       'REACTIVE - ASEAN.1', 'REACTIVE - CANSAC', 'REACTIVE - CISCOREADY','REACTIVE - EMEAR-PILOT','REACTIVE - GROWTH INITIATIVE','REACTIVE - KYLE WATTS','REACTIVE - RENEWALS Q3 KSO','PROACTIVE REFRESH BDM',   ### THE 412 HEALTHCARE REQUESTS
                                       'TEST', 'TEST REQUEST'])

In [168]:
requesters_to_remove_2022 = list(['lsweidan@cisco.com','nenglema@cisco.com','escudder@cisco.com', 'jestjohn@cisco.com' ])

In [169]:
temp_campaigns_to_remove_from_count = list([ #'BCS 3.0 TARGETING HEALTHCARE CAMPAIGN',
                                           'PROACTIVE - RESEARCH',
                                           'REACTIVE - CXIB',
                                           'REACTIVE - GROWTH INITIATIVE',
                                           'TEST REQUEST'])

In [170]:
def remove_from_the_items_to_be_counted(requester_name, compass_campaign_name, SL3, created_date, date_completed):   # Anything FALSE is intended to be removed. #When this gets into Tableau, we only want to select the TRUE values
    if pd.isnull(requester_name) or pd.isnull(date_completed):
        return False
    elif created_date < pd.to_datetime('20210801', format='%Y%m%d') and (requester_name in requesters_to_remove_2021 or compass_campaign_name in campaigns_to_remove_from_count_2021):
        return False
       
    elif created_date >= pd.to_datetime('20210801', format='%Y%m%d') and (requester_name in requesters_to_remove_2022 or compass_campaign_name in temp_campaigns_to_remove_from_count):
        return False
    
    else:
        return True

In [171]:
df['REMOVE_FROM_COUNT'] = df.apply(lambda x: remove_from_the_items_to_be_counted(x['Requester Name'],x['Campaign'], x['Sales Level 3'], x['Create Date'], x['Date Completed']), axis=1)

In [172]:
df.loc[(df['Campaign'].str.contains('IDENTIFIED RISKS', na=False, case=False)) & (df['Sales Level 1']=='AMERICAS') & (df['REMOVE_FROM_COUNT']==True) ].groupby(['Campaign','Sales Level 3','REMOVE_FROM_COUNT']).count()['Request ID']

Campaign                                                 Sales Level 3      REMOVE_FROM_COUNT
BCS 3.0 TARGETING GLOBAL COMPASS IDENTIFIED RISKS (NEW)  ASP TELCO MOBILE   True                  1
                                                         ASP US SEGMENTS    True                 29
                                                         ASP_CANADA         True                  2
                                                         ASP_LATAM          True                  1
                                                         COMMERCIAL CAN     True                  7
                                                         ENTERPRISE CA      True                  8
                                                         GES WEST           True                 69
                                                         PUBLIC SECTOR CAN  True                 15
                                                         SLED-EAST AREA     True                 26
      

In [173]:
df.loc[df['REMOVE_FROM_COUNT'] == True].groupby(['Requester Name','Campaign Type','REMOVE_FROM_COUNT']).count()['Request ID']

Requester Name      Campaign Type     REMOVE_FROM_COUNT
AABUL@CISCO.COM     FIELD             True                  1
AADAY@CISCO.COM     TARGETING         True                  2
AAFRICA@CISCO.COM   FIELD             True                  2
AAKKAWI@CISCO.COM   COVERAGE PROGRAM  True                  1
AALAJAJI@CISCO.COM  TARGETING         True                  6
                                                           ..
YUKOHAYA@CISCO.COM  TARGETING         True                  1
YVWOIT@CISCO.COM    TARGETING         True                  1
ZALGHAMD@CISCO.COM  FIELD             True                  1
ZISTEPHE@CISCO.COM  FIELD             True                 32
ZJANICEL@CISCO.COM  FIELD             True                  1
Name: Request ID, Length: 1775, dtype: int64

In [174]:
df.shape

(42717, 145)

### RR (internal) field, a.k.a. Readout Required:
#### If a record is OLDER than 6 months, a readout will NOT be done.
#### If a record's Compass Campaign Name is in an exclusion list (TBD), a readout will not be done.
#### If either of the above conditions are met, set a "NO" value in the "RR (internal)" field, else set a "YES" value there

In [175]:
campaign_exclusion_list = ['PROACTIVE - REFRESH LIST','REACTIVE - GROWTH INITIATIVE','PROACTIVE - CS RESEARCH','PROACTIVE - IGNORE','PROACTIVE - TAC LEAD','TEST','TEST REQUEST','REACTIVE - BDM','ACCELERATOR','PROACTIVE - PRESALES ALIGN PITSTOP EFFORT','REACTIVE - CISCOREADY','REACTIVE - PARTNER','REACTIVE - PARTNER.1','REACTIVE - PARTNER.2',]

In [176]:
campaign_inclusion_list = ['AMERICAS SP','REACTIVE - UKI – SUZI','REACTIVE','REACTIVE - AOJC','REACTIVE - AMERICAS','BCS TARGETING AMERICAS','BCS TARGETING - UKI','REACTIVE - EMEAR','REACTIVE -GESW','BCS TARGETING AMERICAS-CANADA','BCS 3.0 TARGETING',]

In [177]:
requester_exclusion_list = ['wzeliger@cisco.com','mragam@cisco.com','caltree@cisco.com','obustosm@cisco.com','antchris@cisco.com',
                            'dadiaspe@cisco.com','lsweidan@cisco.com','jforgy@cisco.com','jguggenhe@cisco.com', 'stehartm@cisco.com',
                            'abdulham@cisco.com','pamohan@cisco.com','charpost@cisco.com','javalenc@cisco.com','kkaler@cisco.com']

In [178]:
lookback_date = dt.datetime.now().date() + dt.timedelta(days = -92)

In [179]:
lookback_date

datetime.date(2022, 1, 17)

In [180]:
### Sanity Indicator - Are there records without the Request ID?
df.loc[df['Request ID'].isnull()][['Created By','Customer Name','GUID','Deal ID','Requester Name','Campaign',]] #.groupby(['Created By'])

,Created By,Customer Name,GUID,Deal ID,Requester Name,Campaign


In [181]:
df = df.loc[df['Request ID'].notnull()]

In [182]:
not_in_list = ['test','test request','accelerator','proactive - ignore']

In [183]:
df = df.loc[~df['Campaign'].str.lower().isin(not_in_list)]

In [184]:
# guidtest1 = '6829690,35452880,89286617'
# guidtest2 = '512349548'

In [185]:
# print(guidtest1.split(','), guidtest2.split(','))

In [186]:
### Start with GUID. If GUID is None, use CAV ID, if CAV ID, is None, use CustomerName, if GUID is bad or a list with unacceptable alpha lexicon values, use CustomerName

def make_fake_guid(guid, cavid, customername):
    import pdb
    if pd.isnull(guid):
        if pd.isnull(cavid) or cavid == -999:
            return str(customername[:10])
        else:
            if not isinstance(cavid, int):
                cavid = cavid.split(',')
                return str(cavid[0])
    elif not isinstance(guid,int):
        try:
            if isinstance(guid,float):
                guid = str(int(guid))
                return guid
            
            elif isinstance(guid,str):
                guid = guid.split(',')
                guid = str(guid[0])
                return guid
        except:
            return str(guid)
    else:
        return str(guid)

In [187]:
# make_fake_guid(np.nan, guidtest1, 'Jac')

In [188]:
# ### Start with GUID. If GUID is None, use CAV ID, if CAV ID, is None, use CustomerName, if GUID is bad or a list with unacceptable alpha lexicon values, use CustomerName
# ### OLD METHOD from Smartsheet

# def make_fake_guid(guid, cavid, customername):
#     if pd.isnull(guid):
#         if pd.isnull(cavid):
#             return str(customername[:10])
#         elif isinstance(cavid, float):
#             return str(int(cavid))
#         elif ',' in cavid:
#             quicklist = list(cavid.split(','))
#             return str(int(quicklist[0]))
#         else:
#             return str(int(cavid))
#     elif type(guid) == float:
#         return str(int(guid))
    
#     elif isinstance(guid, str):
#         quicklist = list(cavid.split(','))
#         return str(int(quicklist[0]))
        
#         if int(guid):
#             return str(int(guid))
#         else:
#             return str(customername[:10])
    
#     elif ',' in guid:
#         quicklist = list(guid.split(','))
#         return str(int(quicklist[0]))
    
#     elif ';' in guid:
#         quicklist = list(guid.split(';'))
#         return str(int(quicklist[0]))
    
#     elif '\r\n' in guid:
#         quicklist = list(guid.split('\r\n'))
#         return str(int(quicklist[0]))
    
#     elif guid == '-':
#         return str(customername[:10])
        
#     elif str(guid).isalpha():
#         return str(customername[:10])
    
#     elif ' ' in guid:
#         quicklist = list(guid.split(' '))
#         if quicklist[0].isalpha():
#             return str(customername[:10])
#         else:
#             return str(int(quicklist[0]))
    
#     elif len(str(guid)) > 12:
#         return str(guid)[:12]
    
#     else:
#         return str(guid)

In [189]:
df['FakeID'] = df.apply(lambda x: make_fake_guid(x['GUID'], x['CAV ID'], x['Squeezed']), axis=1)

In [190]:
df[['Request ID','GUID','FakeID']]

,Request ID,GUID,FakeID
0,118405,44138,44138
1,118396,12847,12847
2,118397,39041042,39041042
3,118398,36861,36861
4,118399,4771431,4771431
...,...,...,...
42712,114327,3985794,3985794
42713,114365,347664188,347664188
42714,114344,3098248,3098248
42715,114370,2644050,2644050


In [191]:
def make_hash_entity(prehash_entity, bdm, readoutdate, items_recommded):    #compassurl, 
    entity_to_hash = str(prehash_entity).upper() + '_' + str(bdm).upper() + '_' + str(readoutdate) + '_' + str(items_recommded).upper()   #str(compassurl).upper() + '_' + 
    return entity_to_hash

In [192]:
df['Prehash_Entity'] = df.apply(lambda x: x['Prepend'] + x['FakeID'], axis=1)

In [193]:
df.loc[df['Prehash_Entity'].notnull()][['Request ID','Create Date','Customer Name','GUID','FakeID','CAV ID','Prepend','Prehash_Entity']]

,Request ID,Create Date,Customer Name,GUID,FakeID,CAV ID,Prepend,Prehash_Entity
0,118405,2022-04-07 20:23:50.186,ADOBE SYSTEMS INC RPORATED,44138,44138,NaN,AD,AD44138
1,118396,2022-04-07 20:23:28.513,DILLARD'S INC,12847,12847,NaN,DI,DI12847
2,118397,2022-04-07 20:23:31.109,NEIMAN MARCUS GROUP INC THE,39041042,39041042,NaN,NE,NE39041042
3,118398,2022-04-07 20:23:33.586,ARCTIC SLOPE REGIONAL CORP.,36861,36861,NaN,AR,AR36861
4,118399,2022-04-07 20:23:36.124,ILLUMINA INC,4771431,4771431,NaN,IL,IL4771431
...,...,...,...,...,...,...,...,...
42712,114327,2022-03-03 20:26:19.775,TROY CORPORATION,3985794,3985794,NaN,TR,TR3985794
42713,114365,2022-03-03 20:28:35.054,VALPAK,347664188,347664188,NaN,VA,VA347664188
42714,114344,2022-03-03 20:27:18.596,UNITED RENTALS,3098248,3098248,NaN,UN,UN3098248
42715,114370,2022-03-03 20:28:52.260,VERSAR INC,2644050,2644050,NaN,VE,VE2644050


## Set the Deal ID on the Refresh Record based upon the Previous Request ID

In [194]:
# request_dealid_dict = {}
req_dealids = df[['Request ID','Deal ID']].copy()
req_dealids = req_dealids.dropna(how='any', axis=0)
req_dealids.set_index('Request ID', inplace=True)
request_dealid_dict = req_dealids.to_dict()

In [195]:
# request_dealid_dict['Deal ID']['2021202114287']

In [196]:
def get_the_deal_id(requestid):
    dealid = None
    if requestid in request_dealid_dict['Deal ID']:
        dealid = request_dealid_dict['Deal ID'][requestid]
        if isinstance(dealid, list):
            return dealid
        elif isinstance(dealid, str) and not dealid.isalpha():
            return dealid
        elif isinstance(dealid, float):
            return int(dealid)
        else:
            return dealid
    else:
        return None

In [197]:
# get_the_deal_id('202120211223236')
# df['Prior Deal ID'] = df['Previous Request ID'].apply(get_the_deal_id)

In [198]:
# df.loc[df['Prior Deal ID'].notnull() & df['Compass Campaign Name'].str.contains('close the gap', case=False)][['Request ID','Previous Request ID','Prior Deal ID','Deal ID','Compass Campaign Name']]

In [199]:
df['Entity_to_hash'] = pd.NA

In [200]:
def is_this_a_targeting_campaign(compass_campaign_name):
    if pd.isnull(compass_campaign_name):
        return False
    else:
        compass_campaign_name = str(compass_campaign_name)
    
    if 'major' in compass_campaign_name.lower():
        return True
    elif 'sspt' in compass_campaign_name.lower():
        return True
    elif 'solution' in compass_campaign_name.lower():
        return True
    elif 'success' in compass_campaign_name.lower():
        return True
    elif 'st target' in compass_campaign_name.lower():
        return True
    elif 'ctu' in compass_campaign_name.lower():
        return True
    elif 'bcs' in compass_campaign_name.lower():
        return True
    elif 'partner' in compass_campaign_name.lower():
        return False
    else:
        return False

In [201]:
df['is_Targeting_Campaign'] = df['Campaign'].apply(is_this_a_targeting_campaign)

In [202]:
def set_targeting_campaign_type(compass_campaign_name):
    compass_campaign_name = str(compass_campaign_name)
    
    if 'major' in compass_campaign_name.lower():
        return "MAJORS"
    elif 'sspt' in compass_campaign_name.lower():
        return "SSPT"
    elif 'solution' in compass_campaign_name.lower():
        return "SSPT"
    elif 'success' in compass_campaign_name.lower():
        return "SUCCESSTRACKS"
    elif 'st target' in compass_campaign_name.lower():
        return "SUCCESSTRACKS"
    elif 'ctu' in compass_campaign_name.lower():
        return "SNTC"
    elif 'bcs' in compass_campaign_name.lower():
        return "BCS"
    elif 'partner' in compass_campaign_name.lower():
        return 'PARTNER'
    else:
        return "NON-TARGETING"

In [203]:
df['Targeting Campaign Type'] = df['Campaign'].apply(set_targeting_campaign_type)

In [204]:
# def set_record_type(previous_request_id):
#     if not pd.isnull(previous_request_id):
#         return 'Refreshed'
#     else:
#         return 'Original'

In [205]:
# df['Record Type'] = df['Previous Request ID'].apply(set_record_type)

### We are Hashing the Company ID, the Compass Advisor, the Readout_Date/Scheduled, and the Items Recommended

In [206]:
df.loc[(df['Items Recommended'].notnull()) | ((df['Readout Delivery Status'].notnull()) & (~df['Readout Delivery Status'].isin(['Declined','Postponed']))) & (df['Compass Advisor'].notnull()),'Entity_to_hash'] = \
df.loc[(df['Items Recommended'].notnull()) | ((df['Readout Delivery Status'].notnull()) & (~df['Readout Delivery Status'].isin(['Declined','Postponed']))) & (df['Compass Advisor'].notnull())].apply(lambda x: make_hash_entity(x['Prehash_Entity'], x['Compass Advisor'], x['Readout_Date'], x['Items Recommended']), axis=1)

In [207]:
def hash_the_readout(entity_to_hash):
    #if pd.isnull(entity_to_hash):  #or entity_to_hash == 'NA'
    if not entity_to_hash:
        return pd.NA
    else:
        entity_to_hash = entity_to_hash.encode()
        hashed = hashlib.md5(entity_to_hash)
        hashed = str(hashed.hexdigest())
        return hashed

In [208]:
df['HashedReadout'] = pd.NA

In [209]:
df.loc[df['Entity_to_hash'].notnull(),'HashedReadout'] = df.loc[df['Entity_to_hash'].notnull()].apply(lambda x: hash_the_readout(x['Entity_to_hash']), axis =1)

In [210]:
df.loc[df['Entity_to_hash'].notnull()][['Request ID','HashedReadout']]

,Request ID,HashedReadout
17,118241,7be3ff47ec1111bfdc100c6665e29afd
18,118242,edc895a4ef49857a52d3819bc9f96037
19,118243,a083f50ab9bdfe04d3b465d27c327823
20,118244,a298bf54f519d11b2580950e4a531f70
21,118245,388d6bed783c8690196a61d58e2e0dee
...,...,...
42677,113860,513d11bcecfef5ad2e4dc5863d329c02
42706,114160,98e500603fe4c61d41eff54f79035a7c
42707,114181,6b60675559937df73c031a505cf397e4
42714,114344,fdb760fa68df5fc351377722a95372b5


In [211]:
### Checking for Incomplete Readout Records

df.loc[df['Entity_to_hash'].notnull() & (df['Items Recommended'].notnull() & (df['Readout Delivery Status'].isnull() | df['Readout_Date'].isnull()))]\
[['Request ID','Create Date','Customer Name','GUID','Partner Name','Prehash_Entity','Compass Advisor','Readout_Date','Readout Delivery Status','Items Recommended','Entity_to_hash','HashedReadout']]

,Request ID,Create Date,Customer Name,GUID,Partner Name,Prehash_Entity,Compass Advisor,Readout_Date,Readout Delivery Status,Items Recommended,Entity_to_hash,HashedReadout
33401,115450,2022-03-08 19:33:56.340,GLENCORE,2153907,None,GL2153907,Camila Bernal,2022-04-07,NaN,"BCS,EC-HTOM,EC-KT,SNTC,SSPT,ST",GL2153907_CAMILA BERNAL_2022-04-07 00:00:00_BC...,d61c5d9d00b14e9d72a1f2e1d7deeca0
34552,116485,2022-03-31 20:16:45.565,SEGRA,329372341,None,SE329372341,Term,2022-04-12,NaN,"BCS,EC-HTOM,PR,SNTC,SSPT,ST,SW/E-P","SE329372341_TERM_2022-04-12 00:00:00_BCS,EC-HT...",1a3a9bba3838277ee0dc9737f87e91e3
40154,111982,2022-02-08 15:26:57.003,CIGNA,21214,None,CI21214,Mark Vodka,NaT,Complete,"BCS,DNA,EC-AM,EC-HTOM,EC-KT,IRR,PR,SNTC,SSPT,ST","CI21214_MARK VODKA_NaT_BCS,DNA,EC-AM,EC-HTOM,E...",e3380c98eeb76ee124db34ba5eb36548
41671,112578,2022-02-16 18:34:29.416,AMX BR EMBRATEL,"4521256,167381571,255262967,6833358,3920922,79...",None,AM4521256,Term,2022-04-11,NaN,"SNTC,SSPT,ST,BCS,EC-HTOM","AM4521256_TERM_2022-04-11 00:00:00_SNTC,SSPT,S...",548a83cdd2b059a7794007ed35b10637


In [212]:
# prehash_check = df.loc[(df['Prehash_Entity'].notnull()) & (df['RR (internal)']=='YES') & (df['Campaign Type']=='PROACTIVE') & (df['Lvl1']=='EMEAR-REGION') & (df['Request FY']=='2021')].groupby(['Prehash_Entity','HashedReadout'])[['Request ID']].nunique()

In [213]:
# prehash_check.reset_index(inplace=True)

In [214]:
# prehash_check

In [215]:
# prehash_check.to_excel(r'C:\Users\phsheari\Desktop\CheckData.xlsx', index=False)

In [216]:
df.loc[df['HashedReadout'].notnull()][['Request ID','Customer Name','Entity_to_hash','HashedReadout','Compass Advisor','Readout_Date','Readout Delivery Status','Items Recommended','Campaign Type','RR_INTERNAL']]

,Request ID,Customer Name,Entity_to_hash,HashedReadout,Compass Advisor,Readout_Date,Readout Delivery Status,Items Recommended,Campaign Type,RR_INTERNAL
17,118241,GRANT COUNTY PUD,"GR2638914_MARK VODKA_2022-04-15 00:00:00_BCS,B...",7be3ff47ec1111bfdc100c6665e29afd,Mark Vodka,2022-04-15,Complete,"BCS,BRW,DNA,EC-AM,IRR,PR,SNTC,SSPT,ST",FIELD,YES
18,118242,ALLY_FINANCIAL,AL15550_SANGEETHA SAMPATH_2022-04-12 00:00:00_...,edc895a4ef49857a52d3819bc9f96037,Sangeetha Sampath,2022-04-12,Complete,"BCS,BRW,DNA,EC-HTOM,EC-KT,IRR,SNTC,SSPT,ST",FIELD,YES
19,118243,DTE ENERGY CO,DT186775_SANGEETHA SAMPATH_2022-04-12 00:00:00...,a083f50ab9bdfe04d3b465d27c327823,Sangeetha Sampath,2022-04-12,Complete,"BCS,BRW,DNA,EC-HTOM,EC-KT,IRR,SNTC,SSPT,ST",FIELD,YES
20,118244,EATON,EA210864776_SANGEETHA SAMPATH_2022-04-12 00:00...,a298bf54f519d11b2580950e4a531f70,Sangeetha Sampath,2022-04-12,Complete,"BCS,BRW,DNA,EC-HTOM,EC-KT,IRR,SNTC,SSPT,ST",FIELD,YES
21,118245,GOODYEAR,GO12656_SANGEETHA SAMPATH_2022-04-12 00:00:00_...,388d6bed783c8690196a61d58e2e0dee,Sangeetha Sampath,2022-04-12,Complete,"BCS,BRW,DNA,EC-HTOM,EC-KT,IRR,SNTC,SSPT,ST",FIELD,YES
...,...,...,...,...,...,...,...,...,...,...
42677,113860,LIBERTY HOME CARE LLC,LI4480252_SARA VILLAMARIN_2022-04-11 00:00:00_...,513d11bcecfef5ad2e4dc5863d329c02,Sara Villamarin,2022-04-11,Emailed Recommendations,"BCS,EC-KT,IRR,PR,SNTC,SSPT,ST",COVERAGE PROGRAM,YES
42706,114160,ROCHESTER GENERAL HEALTH SYSTEM,RO24066_SARA VILLAMARIN_2022-04-11 00:00:00_BC...,98e500603fe4c61d41eff54f79035a7c,Sara Villamarin,2022-04-11,Emailed Recommendations,"BCS,DNA,EC-AM,EC-KT,IRR,PR,SNTC,SSPT,ST",COVERAGE PROGRAM,YES
42707,114181,SAVINGS BANK LIFE INSURANCE OF MASSACHUSETTS,SA6297067_SARA VILLAMARIN_2022-04-11 00:00:00_...,6b60675559937df73c031a505cf397e4,Sara Villamarin,2022-04-11,Emailed Recommendations,"BCS,PR,SNTC,SSPT",COVERAGE PROGRAM,YES
42714,114344,UNITED RENTALS,UN3098248_SARA VILLAMARIN_2022-04-11 00:00:00_...,fdb760fa68df5fc351377722a95372b5,Sara Villamarin,2022-04-11,Emailed Recommendations,"BCS,DNA,EC-AM,EC-HTOM,EC-KT,IRR,PR,SNTC,SSPT,ST",COVERAGE PROGRAM,YES


In [217]:
df['Partner Deal'] = df['Deal ID'].apply(lambda x: 'TRUE' if x == -44444 else 'FALSE')

In [218]:
df.loc[df['Deal ID']!=-44444,['Customer Name','Deal ID','Partner Deal','Partner Name']]

,Customer Name,Deal ID,Partner Deal,Partner Name
0,ADOBE SYSTEMS INC RPORATED,44316883,FALSE,None
1,DILLARD'S INC,44350905,FALSE,None
2,NEIMAN MARCUS GROUP INC THE,44538683,FALSE,None
3,ARCTIC SLOPE REGIONAL CORP.,44364041,FALSE,None
4,ILLUMINA INC,44361487,FALSE,None
...,...,...,...,...
42712,TROY CORPORATION,NaN,FALSE,None
42713,VALPAK,NaN,FALSE,None
42714,UNITED RENTALS,NaN,FALSE,None
42715,VERSAR INC,NaN,FALSE,None


# Setup Compass Advisors, Concierge Advisors, Offer Creators (LPM+) standardized names, Vendors, & Reporting Groups

In [219]:
### Run through the BDM field as if it were GUIDs, replacing "/" with "," as separators, then expanding lists; 
###   ...replacing emails with Proper Names, then feeding proper Names as lookups to return the Report Group field values. 
###  ... Accounting for older, pre-2021 values will have to be added to the roll-up list

In [220]:
advisor_path = r'C:\Users\phsheari\Documents\Compass Advisors\Advisor Email Translation.xlsx'
advisors_df = pd.read_excel(advisor_path)
advisors_df['cecid'] = advisors_df['cecid'].apply(lambda x: x if pd.notnull(x) else pd.NA)
advisors_df['email'] = advisors_df['cecid'].apply(lambda x: x + '@cisco.com' if pd.notnull(x) else pd.NA)

In [221]:
# advisors_df.loc[advisors_df['name']=='Farha Diba']

In [222]:
advisors = dict(zip(advisors_df['name'],advisors_df['report_group']))
advisors_email = dict(zip(advisors_df['email'],advisors_df['report_group']))
advisors_names = dict(zip(advisors_df['name'], advisors_df['report_group']))

In [223]:
def get_advisor_report_group(advisor):    #"bdm" is the compass advisor column from the tracker
    if pd.isnull(advisor):
        return pd.NA
    elif ',' in advisor or '/' in advisor:
        advisor = advisor.lower().strip().replace('/',',').strip()
        advisorlist = advisor.split(',')
        advisor = advisorlist[0]
    
    if advisor.lower().endswith('@cisco.com'):
        return advisors_email.get(advisor, 'Other')
    else:
        return advisors.get(advisor, advisors_names.get(advisor, 'Other'))    

In [224]:
df['Report Group'] = df['Compass Advisor'].apply(get_advisor_report_group)

In [225]:
df.groupby(df['Report Group'])['Report Group'].count()

Report Group
Atif Ahmad         3953
Jenee St John      2504
Other              1973
Stacey Edwards    13469
Name: Report Group, dtype: int64

In [226]:
# df.groupby(['Compass Advisor','Report Group'])['Compass Advisor','Report Group'].count()

In [227]:
# df.loc[:,df.columns.str.startswith('O')].columns      #['Projected Booking ($,000)','Target Fiscal Month', 'Target Fiscal Quarter', 'Target Fiscal Year','Opportunity Name','Forecast Stage','Forecast Status']

In [228]:
#forecast_columns_to_drop = ['Target Fiscal Month', 'Target Fiscal Quarter', 'Target Fiscal Year','Forecast Stage','Forecast Status']  #'Projected Booking ($,000)',

In [229]:
#df.drop(columns=forecast_columns_to_drop, inplace=True)

# Build Salesforce Pipeline 

In [230]:
PIPELINE_COLUMNS = ['Request ID', 'Forecast Status', 'Forecast Stage','Service Source', 'Fiscal Month', 'Fiscal Quarter', 'Fiscal Year', 'Fiscal Y-Q', 'Opportunity Value',]

In [231]:
pipeline = pd.read_excel(r'C:\Users\phsheari\Documents\Compass Tableau Data\TrkrPipeline.xlsx', usecols=PIPELINE_COLUMNS)

In [232]:
pipeline.columns

Index(['Request ID', 'Service Source', 'Forecast Stage', 'Forecast Status',
       'Fiscal Month', 'Fiscal Quarter', 'Fiscal Year', 'Fiscal Y-Q',
       'Opportunity Value'],
      dtype='object')

In [233]:
pipeline = pipeline.loc[pipeline['Service Source']=='New'].groupby(['Request ID','Forecast Stage','Forecast Status','Fiscal Month','Fiscal Quarter','Fiscal Year'])[['Opportunity Value']].sum()

In [234]:
pipeline.reset_index(inplace=True)

In [235]:
pipeline['Opportunity Value'] = pipeline['Opportunity Value'].astype(float)

In [236]:
pipeline['Request ID'] = pipeline['Request ID'].astype(str)

In [237]:
pipeline_rename_fields = {'Fiscal Month':'Target Fiscal Month','Fiscal Quarter':'Target Fiscal Quarter','Fiscal Year':'Target Fiscal Year',}

In [238]:
pipeline.rename(columns=pipeline_rename_fields, inplace=True)

In [239]:
replace_na_values = {'Forecast Stage':'','Forecast Status':'','Target Fiscal Month':'','Target Fiscal Quarter':'','Target Fiscal Year':'', 'Opportunity Value':0.0}

In [240]:
df['Request ID'] = df['Request ID'].astype(str)

In [241]:
df = df.merge(pipeline, how='left', on='Request ID')

In [242]:
df.fillna(value=replace_na_values, inplace=True)

In [243]:
df.rename(columns=pipeline_rename_fields, inplace=True)

In [244]:
df.loc[df['Request ID'].isin(finbi_requests), 'Forecast Stage'] = '5 - Closed Won'
df.loc[df['Request ID'].isin(finbi_requests), 'Forecast Status'] = 'Booked'
df.loc[df['Request ID'].isin(finbi_requests), 'WORKING STATUS'] = 'BOOKED'

In [245]:
df.drop(df.loc[df['Request ID']=='undefined'].index, inplace=True)

In [246]:
df['WORKING STATUS'].unique()

array(['RR_SET_NO', 'READOUT PASSED - RESULT PENDING', 'READOUT DONE',
       'DECLINED', 'ASSIGNED', 'SCHEDULED', 'COMPASS UNDELIVERABLE',
       'EXCEPTION_REVIEW', 'BOOKED', 'READOUT POSTPONED', 'UNASSIGNED'],
      dtype=object)

In [247]:
# set_as_date(df, ['Create Date','Date Completed','Readout_Date','Last Updated'])
df['LAST_UPDATED']

0        2022-04-19 13:32 PM Eastern
1        2022-04-19 13:32 PM Eastern
2        2022-04-19 13:32 PM Eastern
3        2022-04-19 13:32 PM Eastern
4        2022-04-19 13:32 PM Eastern
                    ...             
42650    2022-04-19 13:32 PM Eastern
42651    2022-04-19 13:32 PM Eastern
42652    2022-04-19 13:32 PM Eastern
42653    2022-04-19 13:32 PM Eastern
42654    2022-04-19 13:32 PM Eastern
Name: LAST_UPDATED, Length: 42655, dtype: object

In [248]:
df.loc[:,'Create Date'] = df.loc[df['Create Date'].notnull()]['Create Date'].apply(lambda x: x.strftime('%Y-%m-%d')).apply(lambda x: dt.datetime.strptime(x, '%Y-%m-%d'))
df.loc[df['Date Completed'].notnull(),'Date Completed'] = df.loc[df['Date Completed'].notnull()]['Date Completed'].apply(lambda x: x.strftime('%Y-%m-%d')).apply(lambda x: dt.datetime.strptime(x, '%Y-%m-%d'))
df.loc[df['Readout_Date'].notnull(),'Readout_Date'] = df.loc[df['Readout_Date'].notnull()]['Readout_Date'].apply(lambda x: x.strftime('%Y-%m-%d')).apply(lambda x: dt.datetime.strptime(x, '%Y-%m-%d'))
df.loc[df['Modified'].notnull(),'Modified'] = df.loc[df['Modified'].notnull()]['Modified'].apply(lambda x: x.strftime('%Y-%m-%d')).apply(lambda x: dt.datetime.strptime(x, '%Y-%m-%d'))
#df.loc[:,'LAST_UPDATED'] = df.loc[:,'LAST_UPDATED'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M %p') + ' Eastern/NYC') #.apply(lambda x: dt.datetime.strptime(x, '%Y-%m-%d %H:%M %p'))

In [249]:
df.loc[df['Sales Level 1'] == 'APJC__','Sales Level 1'] = 'APJC'

In [250]:
df[['Request ID','Create Date','Date Completed','Readout_Date','Modified','LAST_UPDATED']]

,Request ID,Create Date,Date Completed,Readout_Date,Modified,LAST_UPDATED
0,118405,2022-04-07,2022-04-11,NaT,2022-04-11,2022-04-19 13:32 PM Eastern
1,118396,2022-04-07,2022-04-11,NaT,2022-04-11,2022-04-19 13:32 PM Eastern
2,118397,2022-04-07,2022-04-11,NaT,2022-04-11,2022-04-19 13:32 PM Eastern
3,118398,2022-04-07,2022-04-11,NaT,2022-04-11,2022-04-19 13:32 PM Eastern
4,118399,2022-04-07,2022-04-11,NaT,2022-04-11,2022-04-19 13:32 PM Eastern
...,...,...,...,...,...,...
42650,114327,2022-03-03,2022-03-08,2022-03-30,2022-04-14,2022-04-19 13:32 PM Eastern
42651,114365,2022-03-03,2022-03-08,2022-03-30,2022-04-14,2022-04-19 13:32 PM Eastern
42652,114344,2022-03-03,2022-03-08,2022-04-11,2022-04-14,2022-04-19 13:32 PM Eastern
42653,114370,2022-03-03,2022-03-08,2022-04-11,2022-04-14,2022-04-19 13:32 PM Eastern


In [251]:
def item_recommended_count(item_name, items_recommended):#, items_recommended):
    if pd.isnull(items_recommended):
        return 0
    elif item_name in items_recommended:
        return 1
    else:
        return 0

In [252]:
df['RRT_BCS']  = df['Items Recommended'].apply(lambda x: item_recommended_count('BCS', x))
df['RRT_EC']   = df['Items Recommended'].apply(lambda x: item_recommended_count('EC', x))
df['RRT_IRR']  = df['Items Recommended'].apply(lambda x: item_recommended_count('IRR', x))
df['RRT_DNAC'] = df['Items Recommended'].apply(lambda x: item_recommended_count('DNA', x))
df['RRT_SSPT'] = df['Items Recommended'].apply(lambda x: item_recommended_count('SSPT', x))
df['RRT_ST']   = df['Items Recommended'].apply(lambda x: item_recommended_count('ST', x))
df['RRT_SNTC'] = df['Items Recommended'].apply(lambda x: item_recommended_count('SNTC', x))
df['RRT_PR']   = df['Items Recommended'].apply(lambda x: item_recommended_count('PR', x))
df['RRT_BRW']  = df['Items Recommended'].apply(lambda x: item_recommended_count('BRW', x))
df['RRT_NULL'] = 0
df['RRT_VBDM'] = 0
df['RRT_LPM+'] = 0

In [253]:
df.loc[df['Items Recommended'].isnull(), 'RRT_NULL'] = 1

In [254]:
df.loc[(df['NEXT_STEPS'].str.contains('engage vbdm', na=False, case=False)) | (df['VBDM'].str.contains('yes', na=False, case=False)), 'RRT_VBDM']=1

In [255]:
df.loc[df['NEXT_STEPS'].str.contains('engage lpm+', na=False, case=False), 'RRT_LPM+']=1

In [256]:
df['LPM_OFFERS'] = df['OFFER_CREATOR_OFFER_CREATED'].apply(lambda x: 1 if x == 'Yes' else 0)

In [257]:
df.loc[(df['Readout_Date'].notnull()) & (df['Items Recommended'].notnull()) & (df['OFFER_CREATOR_OFFER_CREATED'].notnull())][['Readout_Date','Items Recommended','VBDM','RRT_BCS','RRT_EC','RRT_IRR','RRT_DNAC','RRT_ST','RRT_SSPT','RRT_SNTC','RRT_PR','RRT_BRW','RRT_VBDM','RRT_LPM+','RRT_NULL','OFFER_CREATOR_OFFER_CREATED','LPM_OFFERS']]

,Readout_Date,Items Recommended,VBDM,RRT_BCS,RRT_EC,RRT_IRR,RRT_DNAC,RRT_ST,RRT_SSPT,RRT_SNTC,RRT_PR,RRT_BRW,RRT_VBDM,RRT_LPM+,RRT_NULL,OFFER_CREATOR_OFFER_CREATED,LPM_OFFERS
18113,2022-01-21,"BCS,DNA,EC-KT,IRR,SNTC,SSPT,ST",NaN,1,1,1,1,1,1,1,0,0,0,0,0,Yes,1
22530,2022-03-07,"BCS,BRW,IRR,SNTC,ST,SSPT",NaN,1,0,1,0,1,1,1,0,1,0,0,0,Yes,1
22541,2022-03-30,"BCS,ST,IRR,SSPT,EC-AM,EC-KT",NaN,1,1,1,0,1,1,0,0,0,0,0,0,Yes,1
24727,2021-11-25,"ST,IRR,SNTC,SSPT,EC-KT,PR",YES,0,1,1,0,1,1,1,1,0,1,0,0,Yes,1
24924,2021-11-10,"BCS,BRW,ST,SNTC,SSPT,PR,IRR,EC-AM,EC-KT",NO,1,1,1,0,1,1,1,1,1,0,0,0,Yes,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41824,2022-04-04,"BCS,DNA,IRR,PR,SNTC,SSPT,ST",NaN,1,0,1,1,1,1,1,1,0,0,0,0,Yes,1
41827,2022-04-04,"BCS,DNA,EC-KT,IRR,PR,SNTC,SSPT,ST",NaN,1,1,1,1,1,1,1,1,0,0,0,0,Yes,1
41830,2022-04-04,"BCS,DNA,IRR,PR,SNTC,ST",NaN,1,0,1,1,1,0,1,1,0,0,0,0,Yes,1
41831,2022-04-04,"BCS,SNTC,PR,ST,IRR",NaN,1,0,1,0,1,0,1,1,0,0,0,0,Yes,1


In [258]:
df['Method'] = None

In [259]:
df['Measure Group'] = 'Operations'

In [260]:
df.loc[df['Campaign Type'].str.contains('coverage at renewal', na=False, case=False),'Method'] = 'M6 (Coverage at Renewal)' #[['Request ID','Create Date','Customer Name','Compass Campaign Name','Campaign Type']]

In [261]:
df.loc[(df['Campaign'] == 'PROACTIVE - TAC LEAD') | (df['Campaign Type'].str.contains('test', na=False, case=False)),'Method'] = 'OMIT'

In [262]:
df.loc[df['Campaign Type'].str.contains('field', na=False, case=False),'Method'] = 'FIELD'

In [263]:
df.loc[df['Campaign Type'].str.contains('targeting', na=False, case=False),'Method'] = 'TARGETING'

In [264]:
df.loc[df['Campaign Type'].str.contains('cxib', na=False, case=False),'Method'] = 'M6 (CXIB)'

In [265]:
df.loc[(df['Campaign Type'].str.contains('missed attach', na=False, case=False)) | 
       (df['Campaign Type'].str.contains('coverage program', na=False, case=False)) | 
       (df['Campaign Type'] == 'TAC') |
       (df['Campaign'] == 'COVERAGE PROGRAM - TELEMETRY') |
       (df['Campaign'] == 'COVERAGE PROGRAM - SNTC TAC LEADS') |
       (df['Campaign'] == 'REACTIVE - ATTACH REPORT') |
       (df['Campaign'] == 'COVERAGE PROGRAM - FIELD REQUEST') |
       (df['Campaign'] == 'COVERAGE PROGRAM - SNTC PROACTIVE ATTACH') |
       (df['Campaign'] == 'COVERAGE PROGRAM - SSPT') |
       ((df['Campaign'] == 'REACTIVE') & (df['PROACTIVE_REPORT_REQUESTED_FLAG'] == True)) |
       (df['Campaign Type'].str.contains('proactive coverage', na=False, case=False)),'Method'] = 'M1 (Coverage Program)'

In [266]:
# df.loc[(df['Campaign Type'].str.contains('missed attach', na=False, case=False)) | 
#        (df['Campaign Type'].str.contains('coverage program', na=False, case=False)) | 
#        (df['Campaign Type'].str.contains('telemetry', na=False, case=False)) |
#        (df['Campaign Type'] == 'TAC') |
#        (df['Campaign'] == 'REACTIVE - ATTACH REPORT') |
#        (df['Campaign'] == 'COVERAGE PROGRAM - FIELD REQUEST') |
#        (df['Campaign'] == 'COVERAGE PROGRAM - SNTC PROACTIVE ATTACH') |
#        ((df['Campaign'] == 'REACTIVE') & (df['PROACTIVE_REPORT_REQUESTED_FLAG'] == True)) |
#        (df['Campaign Type'].str.contains('proactive coverage', na=False, case=False)),'Method'] = 'M1 (Coverage Program)'

In [267]:
df.loc[(df['Campaign Type'].str.contains('proactive', na=False, case=False)) & (df['Campaign'].str.contains('attach', na=False, case=False))][['Request Type','Create Date','Request Type', 'Campaign Type','Campaign','Request FY','Request FQ','Request FM']]

,Request Type,Create Date,Request Type,Campaign Type,Campaign,Request FY,Request FQ,Request FM


In [268]:
df['Campaign Type'].unique()

array(['CXIB', 'FIELD', 'TARGETING', 'PROACTIVE_OLD',
       'NONREPORTING/TESTS', 'COVERAGE PROGRAM', 'TELEMETRY',
       'COVERAGE AT RENEWAL', 'NO CAMPAIGN'], dtype=object)

In [269]:
df['Targeting Campaign Type'].unique()

array(['NON-TARGETING', 'BCS', 'SNTC', 'PARTNER', 'SSPT', 'SUCCESSTRACKS',
       'MAJORS'], dtype=object)

In [270]:
df.loc[(df['Campaign Type']=='COVERAGE PROGRAM') & (df['Campaign']=='COVERAGE PROGRAM - PROACTIVE ATTACH'), 'Campaign'] = 'COVERAGE - SNTC PROACTIVE ATTACH'

In [271]:
def set_offer_focus(campaign_type, target_campaign_type):
    if campaign_type == 'FIELD':
        return 'ANY'
    elif campaign_type == 'CXIB':
        return 'SNTC'
    elif campaign_type == 'COVERAGE AT RENEWAL':
        return 'SNTC'
    elif (campaign_type == 'MISSED ATTACH') or (campaign_type == 'COVERAGE PROGRAM') or (campaign_type == 'PROACTIVE COVERAGE') or (campaign_type == 'TAC') or (campaign_type == 'TELEMETRY'):
        return 'SNTC'
    elif campaign_type == 'TARGETING' and target_campaign_type == 'MAJORS':
        return 'BCS'
    elif campaign_type == 'TARGETING' and target_campaign_type == 'BCS':
        return 'BCS'
    elif campaign_type == 'TARGETING' and target_campaign_type == 'SUCCESSTRACKS':
        return 'ST'
    elif campaign_type == 'TARGETING' and target_campaign_type == 'SSPT':
        return 'SSPT'
    elif campaign_type == 'TARGETING' and target_campaign_type == 'SNTC':
        return 'SNTC'
    else:
        return None

In [272]:
df['Offer Focus'] = df.apply(lambda x: set_offer_focus(x['Campaign Type'], x['Targeting Campaign Type']), axis=1)

In [273]:
df['Value Type'] = 'Actual'

In [274]:
# df.loc[:,df.columns.str.contains('readout', na=False, case=False)]

In [275]:
def apply_fiscal_year(readout_fy, request_fy):
    if isinstance(readout_fy,str):
        if readout_fy.isdigit():
            return readout_fy
    else:
        return request_fy

In [276]:
# def apply_fiscal_year(oc_readout_fy, readout_fy, request_fy):
#     if isinstance(oc_readout_fy, str):
#         if oc_readout_fy.isdigit():
#             return oc_readout_fy
        
#     elif isinstance(readout_fy,str):
#         if readout_fy.isdigit():
#             return readout_fy
#     else:
#         return request_fy

In [277]:
def apply_fiscal_qtr(readout_fq, request_fq):
    if isinstance(readout_fq,str):
        if readout_fq.isdigit():
            return 'Q' + readout_fq
    else:
        return 'Q' + request_fq

In [278]:
# def apply_fiscal_qtr(oc_readout_fq, readout_fq, request_fq):
#     if isinstance(oc_readout_fq, str):
#         if oc_readout_fq.isdigit():
#             return 'Q' + oc_readout_fq
    
#     elif isinstance(readout_fq,str):
#         if readout_fq.isdigit():
#             return 'Q' + readout_fq
#     else:
#         return 'Q' + request_fq

In [279]:
def apply_injection_month(request_fy, request_fm):
    if isinstance(request_fy,str):
        if request_fy.isdigit():
            return ''.join([request_fy,request_fm])
    else:
        return None

In [280]:
def apply_injection_quarter(request_fy, request_fq):
    if isinstance(request_fy,str):
        if request_fy.isdigit():
            return ''.join([request_fy,'-Q',request_fq])
    else:
        return None

In [281]:
# print(list(df.loc[df['Readout FY']]['Readout FY'].unique()))

In [282]:
df['Report FY'] = df.apply(lambda x: apply_fiscal_year(x['Readout FY'], x['Request FY']), axis=1)

In [283]:
df['Report FQ'] = df.apply(lambda x: apply_fiscal_qtr(x['Readout FQ'], x['Request FQ']), axis=1)

In [284]:
df['Injection Month'] = df.apply(lambda x: apply_injection_month(x['Request FY'],x['Request FM']), axis=1)

In [285]:
df['Injection Quarter'] = df.apply(lambda x: apply_injection_quarter(x['Request FY'],x['Request FQ']), axis=1)

In [286]:
df.loc[(df['Campaign'].str.contains('COVERAGE PROGRAM - SNTC TAC LEADS', na=False, case=False)) ][['Method','Campaign Type','Campaign',]].value_counts()

Method                 Campaign Type     Campaign                         
M1 (Coverage Program)  COVERAGE PROGRAM  COVERAGE PROGRAM - SNTC TAC LEADS    113
dtype: int64

In [287]:
df.loc[(df['Report FY']=='2022') & (df['RR_INTERNAL']=='YES') & (df['ALERT_BELOW_25'] == 0) & (df['Method']=='M1 (Coverage Program)') & (df['OPPTY_STG1_CREATED'] == 1)][['Method','Campaign Type','Campaign',]].value_counts() #(df['Report FQ']=='Q2') & 

Method                 Campaign Type     Campaign                                
M1 (Coverage Program)  COVERAGE PROGRAM  COVERAGE PROGRAM - TELEMETRY                1972
                                         COVERAGE PROGRAM - SNTC PROACTIVE ATTACH    1237
                       FIELD             REACTIVE                                     780
                       COVERAGE PROGRAM  COVERAGE PROGRAM - SNTC TAC LEADS             82
                                         COVERAGE PROGRAM - SSPT                       15
dtype: int64

In [288]:
# df.loc[(df['Method']=='FIELD') & (df['Report FY']=='2022')]

In [289]:
df.loc[(df['Report FY']=='2022') & (df['Report FQ']=='Q2') & (df['OPPTY_STG1_CREATED']==1) & (df['Campaign Type'].isin(['FIELD'])) & (df['Method']=='M1 (Coverage Program)')][['Report FY','Request ID','Create Date','Readout_Date','RR_INTERNAL','Compass Advisor','Deal ID','Method','Campaign Type','Campaign','Customer Name','Readout Delivery Status','ALERT_BELOW_25','OPPTY_STG1_CREATED','LPM_PLUS_ENGAGED','PROACTIVE_REPORT_REQUESTED_FLAG','OFFER_CREATED']].to_excel(r'C:\Users\phsheari\Desktop\FIELDS.xlsx', index=False)

In [290]:
df.loc[(df['Campaign Type'].str.contains('field', na=False, case=False) | df['Campaign Type'].str.contains('target', na=False, case=False))  & (df['LPM_PLUS'].str.contains('yes', na=False, case=False) | df['NEXT_STEPS'].str.contains('engage lpm', na=False, case=False)),'Method'] = \
df.loc[(df['Campaign Type'].str.contains('field', na=False, case=False) | df['Campaign Type'].str.contains('target', na=False, case=False))  & (df['LPM_PLUS'].str.contains('yes', na=False, case=False) | df['NEXT_STEPS'].str.contains('engage lpm', na=False, case=False)),'Method'] = 'M1 (Coverage Program)'

In [291]:
df.loc[(df['Report FY']=='2022') & (df['Report FQ']=='Q2') & (df['RR_INTERNAL']=='YES') & (df['Sales Level 1'] == 'AMERICAS') & (df['OPPTY_STG1_CREATED'] == 1) & (df['Method'].str.contains('M1', na=False, case=False)) & (df['Campaign Type'].isin(['FIELD','COVERAGE PROGRAM','PROACTIVE COVERAGE','TAC','TELEMETRY','SSPT','TARGETING'])) & (df['ALERT_BELOW_25'] == 0)]['Campaign Type'].value_counts() #[['Report FY','Request ID','Create Date','Readout_Date','RR_INTERNAL','Compass Advisor','Deal ID','Method','Campaign Type','Campaign','Customer Name','Readout Delivery Status','OPPTY_STG1_CREATED','LPM_READOUTS_DONE','LPM_READOUTS_DECLINED','OFFER_CREATED']]

COVERAGE PROGRAM    686
FIELD               611
TARGETING            22
Name: Campaign Type, dtype: int64

In [292]:
df.loc[(df['Report FY']=='2022') & (df['Report FQ']=='Q2') & (df['ALERT_BELOW_25'] == 0) & (df['RR_INTERNAL']=='YES') & (df['OPPTY_STG1_CREATED'] == 1) & (df['Method'].str.contains('M1', na=False, case=False)) & (df['Campaign Type'].isin(['TARGETING'])) ]['Campaign Type'].value_counts()  #& (df['ALERT_BELOW_25'] == 0) & (df['RR_INTERNAL']=='YES')  & (df['Sales Level 1'] == 'AMERICAS')

TARGETING    33
Name: Campaign Type, dtype: int64

###  ALTER Field and Target requests to M1 Coverage Program Method where LPM Plus = YES

In [293]:
df.loc[(df['Campaign Type'].str.contains('field', na=False, case=False) | df['Campaign Type'].str.contains('target', na=False, case=False)) & (df['LPM_PLUS'].str.contains('yes', na=False, case=False))][['Request ID','Campaign','Campaign Type','Method','LPM_PLUS','NEXT_STEPS']] #  & (df['LPM_PLUS'].str.contains('yes', na=False, case=False))

,Request ID,Campaign,Campaign Type,Method,LPM_PLUS,NEXT_STEPS
24727,24533,BCS 3.0 TARGETING GLOBAL COMPASS IDENTIFIED RI...,TARGETING,M1 (Coverage Program),Yes,"Engage vBDM,Engage Offer Creator"
24923,24465,BCS 3.0 TARGETING GLOBAL COMPASS IDENTIFIED RI...,TARGETING,M1 (Coverage Program),Yes,Engage Offer Creator
24924,24779,BCS 3.0 TARGETING GLOBAL COMPASS IDENTIFIED RI...,TARGETING,M1 (Coverage Program),Yes,Engage Offer Creator
25503,107464,REACTIVE,FIELD,M1 (Coverage Program),Yes,Engage Offer Creator
25504,107427,REACTIVE,FIELD,M1 (Coverage Program),Yes,Engage Offer Creator
...,...,...,...,...,...,...
37127,108719,REACTIVE,FIELD,M1 (Coverage Program),Yes,Engage Offer Creator
37179,108770,REACTIVE,FIELD,M1 (Coverage Program),Yes,Engage Offer Creator
37659,108783,REACTIVE,FIELD,M1 (Coverage Program),Yes,Engage Offer Creator
37666,108781,REACTIVE,FIELD,M1 (Coverage Program),Yes,"No Action,Engage Offer Creator"


In [294]:
df.rename(columns={'COMPASS_READOUTS_DONE':'READOUTS_DONE'}, inplace=True)

In [295]:
opps_stg1 = df.loc[(df['Report FY']=='2022') & (df['ALERT_BELOW_25'] == 0) & (df['RR_INTERNAL']=='YES') & (df['OPPTY_STG1_CREATED'] == 1) & (df['Campaign Type'].isin(['TAC','TARGETING','TELEMETRY','SSPT','FIELD','PROACTIVE COVERAGE','COVERAGE AT RENEWAL','COVERAGE PROGRAM'])) ]   #& (df['Method'].str.contains('M1', na=False, case=False)) 

In [296]:
agg_opps = pd.DataFrame(opps_stg1.groupby(['Method', 'Measure Group','Campaign', 'Campaign Type','Customer Name', 'Sales Level 1', 'Sales Level 2', 'Sales Level 3', 'Sales Level 4','Offer Focus', 'Value Type','Report FY','Report FQ'])['OPPTY_STG1_CREATED'].sum())

In [297]:
agg_opps.reset_index(inplace=True)

In [298]:
agg_opps['Measure'] = 'Stg1 Opportunity'
agg_opps.rename(columns={'OPPTY_STG1_CREATED':'Value'}, inplace=True)
agg_opps

,Method,Measure Group,Campaign,Campaign Type,Customer Name,Sales Level 1,Sales Level 2,Sales Level 3,Sales Level 4,Offer Focus,Value Type,Report FY,Report FQ,Value,Measure
0,FIELD,Operations,REACTIVE,FIELD,(ACSA) AIRPORTS COMPANY OF SOUTH AFRICA,EMEAR-REGION,EMEAR_MEA,CLUSTER_SSA,COUNTRY_ZAF,ANY,Actual,2022,Q2,1,Stg1 Opportunity
1,FIELD,Operations,REACTIVE,FIELD,(PIC) PUBLIC INVESTMENT CORPORATION,EMEAR-REGION,EMEAR_MEA,CLUSTER_SSA,COUNTRY_ZAF,ANY,Actual,2022,Q2,1,Stg1 Opportunity
2,FIELD,Operations,REACTIVE,FIELD,(SAA) SOUTH AFRICAN AIRWAYS,EMEAR-REGION,EMEAR_MEA,CLUSTER_SSA,COUNTRY_ZAF,ANY,Actual,2022,Q2,1,Stg1 Opportunity
3,FIELD,Operations,REACTIVE,FIELD,(SAPS) – SOUTH AFRICAN POLICE SERVICES,EMEAR-REGION,EMEAR_MEA,CLUSTER_SSA,COUNTRY_ZAF,ANY,Actual,2022,Q2,1,Stg1 Opportunity
4,FIELD,Operations,REACTIVE,FIELD,(SARS) SOUTH AFRICAN RESERVE SERVICES,EMEAR-REGION,EMEAR_MEA,SMALL_MEA,SMALL_SSA,ANY,Actual,2022,Q2,1,Stg1 Opportunity
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8222,TARGETING,Operations,SUCCESS TRACKS FOR CAMPUS TARGETING - DYNAMIC,TARGETING,YANTAI WANHUA GROUP,APJC,GREATER_CHINA,CN_NORTH,CN_ENT_NORTH_MET,ST,Actual,2022,Q2,1,Stg1 Opportunity
8223,TARGETING,Operations,SUCCESS TRACKS FOR CAMPUS TARGETING - DYNAMIC,TARGETING,YPFB TRANSPORTE,AMERICAS,LATIN AMERICA,M_C_R,PERU_ANDEAN,ST,Actual,2022,Q1,1,Stg1 Opportunity
8224,TARGETING,Operations,SUCCESS TRACKS FOR CAMPUS TARGETING - DYNAMIC,TARGETING,ZENSAR TECHNOLOGIES LTD,APJC,INDIA_AREA,IND_ENTERPRISE_MAJOR,IND_ENT_STRATEGIC,ST,Actual,2022,Q2,1,Stg1 Opportunity
8225,TARGETING,Operations,SUCCESS TRACKS FOR CAMPUS TARGETING - DYNAMIC,TARGETING,ZURICH INSURANCE GROUP,EMEAR-REGION,EMEAR-CENTRAL,COUNTRY_SWITZERLAND,SWITZERLAND_CGEM,ST,Actual,2022,Q2,1,Stg1 Opportunity


In [299]:
readouts_done = df.loc[(df['Report FY']=='2022') & (df['READOUTS_DONE'] == 1) & (df['Campaign Type'].isin(['TAC','TARGETING','TELEMETRY','SSPT','FIELD','PROACTIVE COVERAGE','COVERAGE AT RENEWAL','COVERAGE PROGRAM'])) ] #& (df['Method'].str.contains('M1', na=False, case=False)) 

In [300]:
agg_readouts = pd.DataFrame(readouts_done.groupby(['Method', 'Measure Group','Campaign', 'Campaign Type','Customer Name', 'Sales Level 1', 'Sales Level 2', 'Sales Level 3', 'Sales Level 4','Offer Focus', 'Value Type','Report FY','Report FQ'])['READOUTS_DONE'].sum())

In [301]:
agg_readouts.reset_index(inplace=True)

In [302]:
agg_readouts['Measure'] = 'Readouts Delivered'
agg_readouts.rename(columns={'READOUTS_DONE':'Value'}, inplace=True)
agg_readouts

,Method,Measure Group,Campaign,Campaign Type,Customer Name,Sales Level 1,Sales Level 2,Sales Level 3,Sales Level 4,Offer Focus,Value Type,Report FY,Report FQ,Value,Measure
0,FIELD,Operations,REACTIVE,FIELD,(ACSA) AIRPORTS COMPANY OF SOUTH AFRICA,EMEAR-REGION,EMEAR_MEA,CLUSTER_SSA,COUNTRY_ZAF,ANY,Actual,2022,Q2,1,Readouts Delivered
1,FIELD,Operations,REACTIVE,FIELD,(PIC) PUBLIC INVESTMENT CORPORATION,EMEAR-REGION,EMEAR_MEA,CLUSTER_SSA,COUNTRY_ZAF,ANY,Actual,2022,Q2,1,Readouts Delivered
2,FIELD,Operations,REACTIVE,FIELD,(SAA) SOUTH AFRICAN AIRWAYS,EMEAR-REGION,EMEAR_MEA,CLUSTER_SSA,COUNTRY_ZAF,ANY,Actual,2022,Q2,1,Readouts Delivered
3,FIELD,Operations,REACTIVE,FIELD,(SAPS) – SOUTH AFRICAN POLICE SERVICES,EMEAR-REGION,EMEAR_MEA,CLUSTER_SSA,COUNTRY_ZAF,ANY,Actual,2022,Q2,1,Readouts Delivered
4,FIELD,Operations,REACTIVE,FIELD,(SARS) SOUTH AFRICAN RESERVE SERVICES,EMEAR-REGION,EMEAR_MEA,SMALL_MEA,SMALL_SSA,ANY,Actual,2022,Q2,1,Readouts Delivered
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4197,TARGETING,Operations,SUCCESS TRACKS FOR CAMPUS TARGETING - DYNAMIC,TARGETING,YAKULT,APJC,ASEAN_AREA,ASEAN_PH,ASEAN_COM_PHL,ST,Actual,2022,Q2,1,Readouts Delivered
4198,TARGETING,Operations,SUCCESS TRACKS FOR CAMPUS TARGETING - DYNAMIC,TARGETING,YAMAHA MOTOR CO. LTD.,APJC,JAPAN__,JP_COMMERCIAL_OTH,JP_COM_SELECT,ST,Actual,2022,Q2,1,Readouts Delivered
4199,TARGETING,Operations,SUCCESS TRACKS FOR CAMPUS TARGETING - DYNAMIC,TARGETING,YANTAI WANHUA GROUP,APJC,GREATER_CHINA,CN_NORTH,CN_ENT_NORTH_MET,ST,Actual,2022,Q2,1,Readouts Delivered
4200,TARGETING,Operations,SUCCESS TRACKS FOR CAMPUS TARGETING - DYNAMIC,TARGETING,ZENSAR TECHNOLOGIES LTD,APJC,INDIA_AREA,IND_ENTERPRISE_MAJOR,IND_ENT_STRATEGIC,ST,Actual,2022,Q2,1,Readouts Delivered


In [303]:
offers_created = df.loc[(df['Report FY']=='2022') & (df['OFFER_CREATED'] == 1) & (df['Campaign Type'].isin(['TAC','TARGETING','TELEMETRY','SSPT','FIELD','PROACTIVE COVERAGE','COVERAGE AT RENEWAL'])) ]   #& (df['Method'].str.contains('M1', na=False, case=False)) 

In [304]:
agg_offers = pd.DataFrame(offers_created.groupby(['Method', 'Measure Group','Campaign', 'Campaign Type','Customer Name', 'Sales Level 1', 'Sales Level 2', 'Sales Level 3', 'Sales Level 4','Offer Focus', 'Value Type','Report FY','Report FQ'])['OFFER_CREATED'].sum())

In [305]:
agg_offers.reset_index(inplace=True)

In [306]:
agg_offers['Measure'] = 'Offers Created'
agg_offers.rename(columns={'OFFER_CREATED':'Value'}, inplace=True)
agg_offers

,Method,Measure Group,Campaign,Campaign Type,Customer Name,Sales Level 1,Sales Level 2,Sales Level 3,Sales Level 4,Offer Focus,Value Type,Report FY,Report FQ,Value,Measure
0,FIELD,Operations,REACTIVE,FIELD,(PIC) PUBLIC INVESTMENT CORPORATION,EMEAR-REGION,EMEAR_MEA,CLUSTER_SSA,COUNTRY_ZAF,ANY,Actual,2022,Q2,1,Offers Created
1,FIELD,Operations,REACTIVE,FIELD,(SAPS) – SOUTH AFRICAN POLICE SERVICES,EMEAR-REGION,EMEAR_MEA,CLUSTER_SSA,COUNTRY_ZAF,ANY,Actual,2022,Q2,1,Offers Created
2,FIELD,Operations,REACTIVE,FIELD,(SARS) SOUTH AFRICAN RESERVE SERVICES,EMEAR-REGION,EMEAR_MEA,SMALL_MEA,SMALL_SSA,ANY,Actual,2022,Q2,1,Offers Created
3,FIELD,Operations,REACTIVE,FIELD,ABSA GROUP,EMEAR-REGION,EMEAR_MEA,CLUSTER_SSA,COUNTRY_ZAF,ANY,Actual,2022,Q2,1,Offers Created
4,FIELD,Operations,REACTIVE,FIELD,ADP LLC,AMERICAS,GLOBAL ENTERPRISE SEGMENT,GES EAST,GLL_K_ADP,ANY,Actual,2022,Q3,1,Offers Created
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1083,TARGETING,Operations,SUCCESS TRACKS FOR CAMPUS TARGETING - DYNAMIC,TARGETING,FERROCARRIL MEXICANO S.A. DE C.V,AMERICAS,LATIN AMERICA,MEXICO-,MEXICO ENTERPRISE,ST,Actual,2022,Q2,1,Offers Created
1084,TARGETING,Operations,SUCCESS TRACKS FOR CAMPUS TARGETING - DYNAMIC,TARGETING,NATIONAL AIR TRAFFIC SERVICES LTD,EMEAR-REGION,EMEAR-UKI,PUBLIC_SECTOR_EAW,PS_EAW_GOVT_NAT_SEC,ST,Actual,2022,Q2,1,Offers Created
1085,TARGETING,Operations,SUCCESS TRACKS FOR CAMPUS TARGETING - DYNAMIC,TARGETING,NEW JERSEY INSTITUTE OF TECHNOLOGY,AMERICAS,US PS MARKET SEGMENT,SLED-EAST AREA,SLED-NORTHEAST OPERATION,ST,Actual,2022,Q2,1,Offers Created
1086,TARGETING,Operations,SUCCESS TRACKS FOR CAMPUS TARGETING - DYNAMIC,TARGETING,NISSAN MEXICANA,AMERICAS,LATIN AMERICA,MEXICO-,MEXICO ENTERPRISE,ST,Actual,2022,Q2,1,Offers Created


In [307]:
uncovered_pipeline = df.loc[(df['Report FY']=='2022') & (df['UNCOVERED_OPPTY_VALUE'] > 0) & (df['Campaign Type'].isin(['TAC','TARGETING','TELEMETRY','SSPT','FIELD','PROACTIVE COVERAGE','COVERAGE AT RENEWAL'])) ]    #& (df['Method'].str.contains('M1', na=False, case=False)) 

In [308]:
agg_unc_pipeline = pd.DataFrame(uncovered_pipeline.groupby(['Method', 'Measure Group','Campaign', 'Campaign Type','Customer Name', 'Sales Level 1', 'Sales Level 2', 'Sales Level 3', 'Sales Level 4','Offer Focus', 'Value Type','Report FY','Report FQ'])['UNCOVERED_OPPTY_VALUE'].sum())

In [309]:
agg_unc_pipeline.reset_index(inplace=True)

In [310]:
agg_unc_pipeline['Measure'] = 'Uncovered Opp Value $(000''s)'
agg_unc_pipeline.rename(columns={'UNCOVERED_OPPTY_VALUE':'Value'},inplace=True)
agg_unc_pipeline

,Method,Measure Group,Campaign,Campaign Type,Customer Name,Sales Level 1,Sales Level 2,Sales Level 3,Sales Level 4,Offer Focus,Value Type,Report FY,Report FQ,Value,Measure
0,FIELD,Operations,REACTIVE,FIELD,(SAPS) – SOUTH AFRICAN POLICE SERVICES,EMEAR-REGION,EMEAR_MEA,CLUSTER_SSA,COUNTRY_ZAF,ANY,Actual,2022,Q2,331.2,Uncovered Opp Value $(000s)
1,FIELD,Operations,REACTIVE,FIELD,(SARS) SOUTH AFRICAN RESERVE SERVICES,EMEAR-REGION,EMEAR_MEA,SMALL_MEA,SMALL_SSA,ANY,Actual,2022,Q2,28.8,Uncovered Opp Value $(000s)
2,FIELD,Operations,REACTIVE,FIELD,ABSA GROUP,EMEAR-REGION,EMEAR_MEA,CLUSTER_SSA,COUNTRY_ZAF,ANY,Actual,2022,Q2,55.8,Uncovered Opp Value $(000s)
3,FIELD,Operations,REACTIVE,FIELD,ALBANY MEDICAL CENTER,AMERICAS,US PS MARKET SEGMENT,SLED-EAST AREA,SLED-NORTHEAST OPERATION,ANY,Actual,2022,Q3,1.2,Uncovered Opp Value $(000s)
4,FIELD,Operations,REACTIVE,FIELD,ALFA,AMERICAS,LATIN AMERICA,MEXICO-,MEXICO COMMERCIAL SELECT,ANY,Actual,2022,Q3,12.0,Uncovered Opp Value $(000s)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
854,TARGETING,Operations,SUCCESS TRACKS FOR CAMPUS TARGETING - DYNAMIC,TARGETING,FERROCARRIL MEXICANO S.A. DE C.V,AMERICAS,LATIN AMERICA,MEXICO-,MEXICO ENTERPRISE,ST,Actual,2022,Q2,19.8,Uncovered Opp Value $(000s)
855,TARGETING,Operations,SUCCESS TRACKS FOR CAMPUS TARGETING - DYNAMIC,TARGETING,NATIONAL AIR TRAFFIC SERVICES LTD,EMEAR-REGION,EMEAR-UKI,PUBLIC_SECTOR_EAW,PS_EAW_GOVT_NAT_SEC,ST,Actual,2022,Q2,10.8,Uncovered Opp Value $(000s)
856,TARGETING,Operations,SUCCESS TRACKS FOR CAMPUS TARGETING - DYNAMIC,TARGETING,NEW JERSEY INSTITUTE OF TECHNOLOGY,AMERICAS,US PS MARKET SEGMENT,SLED-EAST AREA,SLED-NORTHEAST OPERATION,ST,Actual,2022,Q2,31.2,Uncovered Opp Value $(000s)
857,TARGETING,Operations,SUCCESS TRACKS FOR CAMPUS TARGETING - DYNAMIC,TARGETING,NISSAN MEXICANA,AMERICAS,LATIN AMERICA,MEXICO-,MEXICO ENTERPRISE,ST,Actual,2022,Q2,192.0,Uncovered Opp Value $(000s)


In [311]:
group_of_measures = pd.concat([agg_opps,agg_readouts,agg_offers,agg_unc_pipeline], axis=1)

In [312]:
group_of_measures

,Method,Measure Group,Campaign,Campaign Type,Customer Name,Sales Level 1,Sales Level 2,Sales Level 3,Sales Level 4,Offer Focus,...,Sales Level 1,Sales Level 2,Sales Level 3,Sales Level 4,Offer Focus,Value Type,Report FY,Report FQ,Value,Measure
0,FIELD,Operations,REACTIVE,FIELD,(ACSA) AIRPORTS COMPANY OF SOUTH AFRICA,EMEAR-REGION,EMEAR_MEA,CLUSTER_SSA,COUNTRY_ZAF,ANY,...,EMEAR-REGION,EMEAR_MEA,CLUSTER_SSA,COUNTRY_ZAF,ANY,Actual,2022,Q2,331.2,Uncovered Opp Value $(000s)
1,FIELD,Operations,REACTIVE,FIELD,(PIC) PUBLIC INVESTMENT CORPORATION,EMEAR-REGION,EMEAR_MEA,CLUSTER_SSA,COUNTRY_ZAF,ANY,...,EMEAR-REGION,EMEAR_MEA,SMALL_MEA,SMALL_SSA,ANY,Actual,2022,Q2,28.8,Uncovered Opp Value $(000s)
2,FIELD,Operations,REACTIVE,FIELD,(SAA) SOUTH AFRICAN AIRWAYS,EMEAR-REGION,EMEAR_MEA,CLUSTER_SSA,COUNTRY_ZAF,ANY,...,EMEAR-REGION,EMEAR_MEA,CLUSTER_SSA,COUNTRY_ZAF,ANY,Actual,2022,Q2,55.8,Uncovered Opp Value $(000s)
3,FIELD,Operations,REACTIVE,FIELD,(SAPS) – SOUTH AFRICAN POLICE SERVICES,EMEAR-REGION,EMEAR_MEA,CLUSTER_SSA,COUNTRY_ZAF,ANY,...,AMERICAS,US PS MARKET SEGMENT,SLED-EAST AREA,SLED-NORTHEAST OPERATION,ANY,Actual,2022,Q3,1.2,Uncovered Opp Value $(000s)
4,FIELD,Operations,REACTIVE,FIELD,(SARS) SOUTH AFRICAN RESERVE SERVICES,EMEAR-REGION,EMEAR_MEA,SMALL_MEA,SMALL_SSA,ANY,...,AMERICAS,LATIN AMERICA,MEXICO-,MEXICO COMMERCIAL SELECT,ANY,Actual,2022,Q3,12.0,Uncovered Opp Value $(000s)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8222,TARGETING,Operations,SUCCESS TRACKS FOR CAMPUS TARGETING - DYNAMIC,TARGETING,YANTAI WANHUA GROUP,APJC,GREATER_CHINA,CN_NORTH,CN_ENT_NORTH_MET,ST,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8223,TARGETING,Operations,SUCCESS TRACKS FOR CAMPUS TARGETING - DYNAMIC,TARGETING,YPFB TRANSPORTE,AMERICAS,LATIN AMERICA,M_C_R,PERU_ANDEAN,ST,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8224,TARGETING,Operations,SUCCESS TRACKS FOR CAMPUS TARGETING - DYNAMIC,TARGETING,ZENSAR TECHNOLOGIES LTD,APJC,INDIA_AREA,IND_ENTERPRISE_MAJOR,IND_ENT_STRATEGIC,ST,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8225,TARGETING,Operations,SUCCESS TRACKS FOR CAMPUS TARGETING - DYNAMIC,TARGETING,ZURICH INSURANCE GROUP,EMEAR-REGION,EMEAR-CENTRAL,COUNTRY_SWITZERLAND,SWITZERLAND_CGEM,ST,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [313]:
#df.loc[(df['Report FY']=='2022') ].groupby(['Method', 'Measure Group','Campaign', 'Campaign Type','Customer Name', 'Sales Level 1', 'Sales Level 2', 'Sales Level 3', 'Sales Level 4','Offer Focus', 'Value Type','Report FY','Report FQ'])['OPPTY_STG1_CREATED','READOUTS_DONE', 'OFFER_CREATED','UNCOVERED_OPPTY_VALUE'].sum().stack()
agg_rpt = df.loc[(df['Report FY']=='2022') ].groupby(['Method', 'Measure Group','Campaign', 'Campaign Type','Customer Name', 'Sales Level 1', 'Sales Level 2', 'Sales Level 3', 'Sales Level 4','Offer Focus', 'Value Type','Report FY','Report FQ'])['OPPTY_STG1_CREATED','READOUTS_DONE', 'OFFER_CREATED','UNCOVERED_OPPTY_VALUE'].sum().stack()

<ipython-input-313-d1763e190e15>:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  agg_rpt = df.loc[(df['Report FY']=='2022') ].groupby(['Method', 'Measure Group','Campaign', 'Campaign Type','Customer Name', 'Sales Level 1', 'Sales Level 2', 'Sales Level 3', 'Sales Level 4','Offer Focus', 'Value Type','Report FY','Report FQ'])['OPPTY_STG1_CREATED','READOUTS_DONE', 'OFFER_CREATED','UNCOVERED_OPPTY_VALUE'].sum().stack()


In [314]:
agg_report = pd.DataFrame(agg_rpt.reset_index()).rename(columns={'level_13':'Measure',0:'Value'})

In [315]:
agg_report.loc[agg_report['Measure']=='UNCOVERED_OPPTY_VALUE']    #.pivot_table(index=['Measure Group','Measure SubGroup', 'Sales Level 1','Measure'], columns=['Report FY','Report FQ','Value Type'], values='Value')

,Method,Measure Group,Campaign,Campaign Type,Customer Name,Sales Level 1,Sales Level 2,Sales Level 3,Sales Level 4,Offer Focus,Value Type,Report FY,Report FQ,Measure,Value
3,FIELD,Operations,REACTIVE,FIELD,(ACSA) AIRPORTS COMPANY OF SOUTH AFRICA,EMEAR-REGION,EMEAR_MEA,CLUSTER_SSA,COUNTRY_ZAF,ANY,Actual,2022,Q2,UNCOVERED_OPPTY_VALUE,0.0
7,FIELD,Operations,REACTIVE,FIELD,(PIC) PUBLIC INVESTMENT CORPORATION,EMEAR-REGION,EMEAR_MEA,CLUSTER_SSA,COUNTRY_ZAF,ANY,Actual,2022,Q2,UNCOVERED_OPPTY_VALUE,0.0
11,FIELD,Operations,REACTIVE,FIELD,(SAA) SOUTH AFRICAN AIRWAYS,EMEAR-REGION,EMEAR_MEA,CLUSTER_SSA,COUNTRY_ZAF,ANY,Actual,2022,Q2,UNCOVERED_OPPTY_VALUE,0.0
15,FIELD,Operations,REACTIVE,FIELD,(SAPS) – SOUTH AFRICAN POLICE SERVICES,EMEAR-REGION,EMEAR_MEA,CLUSTER_SSA,COUNTRY_ZAF,ANY,Actual,2022,Q2,UNCOVERED_OPPTY_VALUE,331.2
19,FIELD,Operations,REACTIVE,FIELD,(SARS) SOUTH AFRICAN RESERVE SERVICES,EMEAR-REGION,EMEAR_MEA,CLUSTER_SSA,COUNTRY_ZAF,ANY,Actual,2022,Q2,UNCOVERED_OPPTY_VALUE,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79887,TARGETING,Operations,SUCCESS TRACKS FOR CAMPUS TARGETING - DYNAMIC,TARGETING,ZIMPLATS HOLDINGS LIMITED (ZIMPLATS),EMEAR-REGION,EMEAR_MEA,SMALL_MEA,CLUSTER_VELOCITY,ST,Actual,2022,Q1,UNCOVERED_OPPTY_VALUE,0.0
79891,TARGETING,Operations,SUCCESS TRACKS FOR CAMPUS TARGETING - DYNAMIC,TARGETING,ZLM VERZEKERINGEN,EMEAR-REGION,EMEAR-NORTH,COUNTRY_NETHERLANDS,COMMERCIAL_NLD,ST,Actual,2022,Q1,UNCOVERED_OPPTY_VALUE,0.0
79895,TARGETING,Operations,SUCCESS TRACKS FOR CAMPUS TARGETING - DYNAMIC,TARGETING,ZURICH INSURANCE GROUP,EMEAR-REGION,EMEAR-CENTRAL,COUNTRY_SWITZERLAND,SWITZERLAND_CGEM,ST,Actual,2022,Q2,UNCOVERED_OPPTY_VALUE,0.0
79899,TARGETING,Operations,SUCCESS TRACKS FOR CAMPUS TARGETING - DYNAMIC,TARGETING,ZWEITES DEUTSCHES FERNSEHEN GEMEINNUTZIGE ANST...,EMEAR-REGION,EMEAR_SP,SP_NORTH,MEDIA_GROUP,ST,Actual,2022,Q3,UNCOVERED_OPPTY_VALUE,0.0


In [316]:
agg_report.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79904 entries, 0 to 79903
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Method         79904 non-null  object 
 1   Measure Group  79904 non-null  object 
 2   Campaign       79904 non-null  object 
 3   Campaign Type  79904 non-null  object 
 4   Customer Name  79904 non-null  object 
 5   Sales Level 1  79904 non-null  object 
 6   Sales Level 2  79904 non-null  object 
 7   Sales Level 3  79904 non-null  object 
 8   Sales Level 4  79904 non-null  object 
 9   Offer Focus    79904 non-null  object 
 10  Value Type     79904 non-null  object 
 11  Report FY      79904 non-null  object 
 12  Report FQ      79904 non-null  object 
 13  Measure        79904 non-null  object 
 14  Value          79904 non-null  float64
dtypes: float64(1), object(14)
memory usage: 9.1+ MB


In [317]:
target_dtypes = {'Report FY': str, 'Value':np.float64}

In [318]:
targets = pd.read_excel(r'C:\Users\phsheari\Documents\Compass SNOWFLAKE\data_download\Mock_Dashboard Targets_SLTest.xlsx', sheet_name = 'Targets', dtype = target_dtypes, na_values = 0.0)

In [319]:
# targets = targets.loc[targets['Measure']!='DATA_PACKAGE_CREATED']

In [320]:
targets.loc[targets['Offer Focus']=='SNTC, PSS', 'Offer Focus'] = 'SNTC'

In [321]:
targets.loc[targets['Value Type']=='Actual']

,Method,Measure Group,Campaign,Campaign Type,Customer Name,Sales Level 1,Sales Level 2,Sales Level 3,Sales Level 4,Offer Focus,Value Type,Report FY,Report FQ,Measure,Value,Unnamed: 15
384,M1 (Coverage Program),Operations,COVERAGE PROGRAM - SNTC PROACTIVE ATTACH,PROACTIVE COVERAGE,NaN,AMERICAS,NaN,NaN,NaN,SNTC,Actual,2022,Q1,OPPTY_STG1_CREATED,6.000,NaN
385,M1 (Coverage Program),Operations,COVERAGE PROGRAM - SNTC PROACTIVE ATTACH,PROACTIVE COVERAGE,NaN,AMERICAS,NaN,NaN,NaN,SNTC,Actual,2022,Q1,READOUTS_DONE,30.000,NaN
386,M1 (Coverage Program),Operations,COVERAGE PROGRAM - SNTC PROACTIVE ATTACH,PROACTIVE COVERAGE,NaN,AMERICAS,NaN,NaN,NaN,SNTC,Actual,2022,Q1,OFFER_CREATED,23.000,NaN
387,M1 (Coverage Program),Operations,COVERAGE PROGRAM - SNTC PROACTIVE ATTACH,PROACTIVE COVERAGE,NaN,AMERICAS,NaN,NaN,NaN,SNTC,Actual,2022,Q1,UNCOVERED_OPPTY_VALUE,4776.000,NaN
388,M1 (Coverage Program),Bookings-New,COVERAGE PROGRAM - SNTC PROACTIVE ATTACH,PROACTIVE COVERAGE,NaN,AMERICAS,NaN,NaN,NaN,SNTC,Actual,2022,Q1,Incr ARR $(000s),8000.000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
793,M1 (Coverage Program),Bookings-New,SUCCESS TRACKS FOR CAMPUS TARGETING - DYNAMIC,TARGETING,NaN,EMEAR-REGION,NaN,NaN,NaN,SNTC,Actual,2022,Q2,Incr ARR $(000s),3.242,NaN
794,M1 (Coverage Program),Bookings-New,SUCCESS TRACKS FOR CAMPUS TARGETING - DYNAMIC,TARGETING,NaN,AMERICAS,NaN,NaN,NaN,SNTC,Actual,2022,Q3,Incr ARR $(000s),20.225,NaN
795,M1 (Coverage Program),Bookings-New,SUCCESS TRACKS DYNAMIC CAMPAIGN,TARGETING,NaN,AMERICAS,NaN,NaN,NaN,SNTC,Actual,2022,Q3,Incr ARR $(000s),161.155,NaN
796,M1 (Coverage Program),Bookings-New,BCS 3.0 TARGETING GLOBAL COMPASS IDENTIFIED RI...,TARGETING,NaN,AMERICAS,NaN,NaN,NaN,SNTC,Actual,2022,Q3,Incr ARR $(000s),6.242,NaN


In [322]:
targets['Method'].unique()

array(['M1 (Coverage Program)', 'M6 (Coverage at Renewal)'], dtype=object)

In [323]:
opp_flow_dash = pd.concat([agg_report, targets]).fillna(0.0)

In [324]:
opp_flow_dash.loc[(opp_flow_dash['Measure Group'].str.contains('operations', na=False, case=False))].pivot_table(index=['Method','Measure Group', 'Sales Level 1','Sales Level 2','Sales Level 3','Sales Level 4','Measure'], columns=['Report FY','Report FQ','Value Type'], values='Value', aggfunc='sum', margins=False).fillna(0.0)

Report FY                                                                                                   2022  \
Report FQ                                                                                                     Q1   
Value Type                                                                                                Actual   
Method    Measure Group Sales Level 1 Sales Level 2 Sales Level 3    Sales Level 4  Measure                        
FIELD     Operations    AMERICAS      AMERICAS_SP   ASP CSP          ASP ATT        OFFER_CREATED            0.0   
                                                                                    OPPTY_STG1_CREATED       0.0   
                                                                                    READOUTS_DONE            0.0   
                                                                                    UNCOVERED_OPPTY_VALUE    0.0   
                                                    ASP TELCO MOBILE ASP AMW        OFFER_CREATED            0.0   
...                                                                                                          ...   
TARGETING Operations    EMEAR-REGION  EMEAR_SP      SP_VDF_LG        LIBERTY_GLOBAL UNCOVERED_OPPTY_VALUE    0.0   
                                                                     VODAFONE_      OFFER_CREATED            0.0   
                                                                                    OPPTY_STG1_CREATED       0.0   
                                                                                    READOUTS_DONE            0.0   
                                                                                    UNCOVERED_OPPTY_VALUE    0.0   

Report FY                                                                                                         \
Report FQ                                                                                                          
Value Type                                                                                                Target   
Method    Measure Group Sales Level 1 Sales Level 2 Sales Level 3    Sales Level 4  Measure                        
FIELD     Operations    AMERICAS      AMERICAS_SP   ASP CSP          ASP ATT        OFFER_CREATED            0.0   
                                                                                    OPPTY_STG1_CREATED       0.0   
                                                                                    READOUTS_DONE            0.0   
                                                                                    UNCOVERED_OPPTY_VALUE    0.0   
                                                    ASP TELCO MOBILE ASP AMW        OFFER_CREATED            0.0   
...                                                                                                          ...   
TARGETING Operations    EMEAR-REGION  EMEAR_SP      SP_VDF_LG        LIBERTY_GLOBAL UNCOVERED_OPPTY_VALUE    0.0   
                                                                     VODAFONE_      OFFER_CREATED            0.0   
                                                                                    OPPTY_STG1_CREATED       0.0   
                                                                                    READOUTS_DONE            0.0   
                                                                                    UNCOVERED_OPPTY_VALUE    0.0   

Report FY                                                                                                         \
Report FQ                                                                                                     Q2   
Value Type                                                                                                Actual   
Method    Measure Group Sales Level 1 Sales Level 2 Sales Level 3    Sales Level 4  Measure                        
FIELD     Operations    AMERICAS      AMERICAS_SP   AS

In [325]:
targets['Measure Group'].unique()

array(['Operations', 'Bookings-New', 'Bookings-Upsell'], dtype=object)

In [326]:
def set_campaign_product_focus(campaign_type, campaign_name, product_type):
    if campaign_type.upper() == 'FIELD' and product_type.upper() == 'SNTC':
        return 'SNTC'
    elif campaign_type.upper() == 'FIELD' and 'BCS' in product_type.upper():
        return 'BCS'
    elif campaign_type.upper() == 'FIELD' and product_type.upper() != 'SNTC' and 'BCS' not in product_type.upper():
        return product_type 
    elif campaign_type.upper() == 'TARGETING':
        if 'BCS' in campaign_name:
            return 'BCS'
        elif 'ST' in campaign_name or 'SUCCESS' in campaign_name:
            return 'ST'
        elif 'SSPT' in campaign_name or 'SOLUTION' in campaign_name:
            return 'SSPT'
        elif 'CTU' in campaign_name:
            return 'SNTC'
    elif (campaign_type.upper() == 'MISSED ATTACH' or campaign_type.upper() == 'COVERAGE PROGRAM'):
        return 'Incr ARR $(000''s)'
    elif campaign_type.upper() == 'COVERAGE AT RENEWAL':
        return 'Incr ARR $(000s)'

In [327]:
def get_lpm_plus_value(request_id):
    #import pdb
    #pdb.set_trace()
    lpm_plus_value = None
    try:
        if df.loc[df['Request ID'] == str(request_id),'LPM_PLUS'].values[0]:
            lpm_plus_value = df.loc[df['Request ID'] == str(request_id),'LPM_PLUS'].values[0]
            if isinstance(lpm_plus_value, float):
                lpm_plus_value = lpm_plus_value.astype(str)
                
    except:
        lpm_plus_value = 'UNKNOWN'
    
    return lpm_plus_value.upper()

In [328]:
def set_method(campaign_type, request_id):
    lpm_plus_value = get_lpm_plus_value(request_id)
    
    if isinstance(lpm_plus_value, str):
        if lpm_plus_value.upper() == 'YES' and campaign_type.upper() == 'COVERAGE AT RENEWAL':
            return 'M6 (Coverage at Renewal)'
    
        elif lpm_plus_value.upper() == 'YES' and campaign_type.upper() != 'COVERAGE AT RENEWAL':
            return 'M1 (Coverage Program)'
    
    elif campaign_type.upper() == 'FIELD':
        return 'FIELD'
    
    elif campaign_type.upper() == 'TARGETING':    
        return 'TARGETING'

In [329]:
df.loc[df['Request ID']=='985','LPM_PLUS']

1139    NaN
Name: LPM_PLUS, dtype: object

In [330]:
# get_lpm_plus_value(105803)

In [331]:
def set_measure_group(campaign_type, campaign_name, service_source, offer_focus, request_id):
    if request_id:
        lpm_plus_value = get_lpm_plus_value(request_id)
    
    if pd.notnull(lpm_plus_value) or lpm_plus_value != np.nan():
        if pd.isnull(service_source) or service_source == 'NA' or service_source == 'N/A' or service_source.upper() == 'UNKNOWN' or service_source == '':
            del lpm_plus_value
            return 'Bookings-Unknown'
        if campaign_type.upper() == 'FIELD' and service_source.upper() == 'NEW' and lpm_plus_value.upper() != 'YES':
            del lpm_plus_value
            return 'Bookings-New'
        elif campaign_type.upper() == 'TARGETING' and service_source.upper() == 'NEW' and lpm_plus_value.upper() != 'YES':
            del lpm_plus_value
            return 'Bookings-New'
        elif (campaign_type.upper() == 'MISSED ATTACH' or campaign_type.upper() == 'COVERAGE PROGRAM') and offer_focus.upper() == 'SNTC':
            del lpm_plus_value
            return 'Bookings-New'
        elif campaign_type.upper() == 'COVERAGE AT RENEWAL' and offer_focus.upper( ) == 'SNTC':
            del lpm_plus_value
            return 'Bookings-New'
        else:
            return 'Bookings-Upsell'
    elif campaign_type.upper() == 'FIELD' and service_source.upper() == 'NEW':
        return 'Bookings-New'
    elif campaign_type.upper() == 'TARGETING' and service_source.upper() == 'NEW':
        return 'Bookings-New'
    elif (campaign_type.upper() == 'MISSED ATTACH' or campaign_type.upper() == 'COVERAGE PROGRAM') and offer_focus.upper() == 'SNTC':
        return 'Bookings-New'
    elif campaign_type.upper() == 'COVERAGE AT RENEWAL' and offer_focus.upper( ) == 'SNTC':
        return 'Bookings-New'
    else:
        return 'Bookings-Upsell'

In [332]:
opp_flow_dash[['Sales Level 1', 'Sales Level 2', 'Sales Level 3', 'Sales Level 4', 'Campaign', 'Campaign Type','Customer Name','Report FY', 'Report FQ', 'Offer Focus', 'Method', 'Measure Group', 'Measure', 'Value Type', 'Value']]

,Sales Level 1,Sales Level 2,Sales Level 3,Sales Level 4,Campaign,Campaign Type,Customer Name,Report FY,Report FQ,Offer Focus,Method,Measure Group,Measure,Value Type,Value
0,EMEAR-REGION,EMEAR_MEA,CLUSTER_SSA,COUNTRY_ZAF,REACTIVE,FIELD,(ACSA) AIRPORTS COMPANY OF SOUTH AFRICA,2022,Q2,ANY,FIELD,Operations,OPPTY_STG1_CREATED,Actual,1.000
1,EMEAR-REGION,EMEAR_MEA,CLUSTER_SSA,COUNTRY_ZAF,REACTIVE,FIELD,(ACSA) AIRPORTS COMPANY OF SOUTH AFRICA,2022,Q2,ANY,FIELD,Operations,READOUTS_DONE,Actual,1.000
2,EMEAR-REGION,EMEAR_MEA,CLUSTER_SSA,COUNTRY_ZAF,REACTIVE,FIELD,(ACSA) AIRPORTS COMPANY OF SOUTH AFRICA,2022,Q2,ANY,FIELD,Operations,OFFER_CREATED,Actual,0.000
3,EMEAR-REGION,EMEAR_MEA,CLUSTER_SSA,COUNTRY_ZAF,REACTIVE,FIELD,(ACSA) AIRPORTS COMPANY OF SOUTH AFRICA,2022,Q2,ANY,FIELD,Operations,UNCOVERED_OPPTY_VALUE,Actual,0.000
4,EMEAR-REGION,EMEAR_MEA,CLUSTER_SSA,COUNTRY_ZAF,REACTIVE,FIELD,(PIC) PUBLIC INVESTMENT CORPORATION,2022,Q2,ANY,FIELD,Operations,OPPTY_STG1_CREATED,Actual,1.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
793,EMEAR-REGION,0.0,0.0,0.0,SUCCESS TRACKS FOR CAMPUS TARGETING - DYNAMIC,TARGETING,0.0,2022,Q2,SNTC,M1 (Coverage Program),Bookings-New,Incr ARR $(000s),Actual,3.242
794,AMERICAS,0.0,0.0,0.0,SUCCESS TRACKS FOR CAMPUS TARGETING - DYNAMIC,TARGETING,0.0,2022,Q3,SNTC,M1 (Coverage Program),Bookings-New,Incr ARR $(000s),Actual,20.225
795,AMERICAS,0.0,0.0,0.0,SUCCESS TRACKS DYNAMIC CAMPAIGN,TARGETING,0.0,2022,Q3,SNTC,M1 (Coverage Program),Bookings-New,Incr ARR $(000s),Actual,161.155
796,AMERICAS,0.0,0.0,0.0,BCS 3.0 TARGETING GLOBAL COMPASS IDENTIFIED RI...,TARGETING,0.0,2022,Q3,SNTC,M1 (Coverage Program),Bookings-New,Incr ARR $(000s),Actual,6.242


In [333]:
opp_flow_dash.pivot_table(index=['Method','Measure Group', 'Sales Level 1','Sales Level 2','Sales Level 3','Sales Level 4','Measure'], columns=['Report FY','Report FQ','Value Type'], values='Value', aggfunc='sum', margins=False).fillna(0.0)

Report FY                                                                                                   2022  \
Report FQ                                                                                                     Q1   
Value Type                                                                                                Actual   
Method    Measure Group Sales Level 1 Sales Level 2 Sales Level 3    Sales Level 4  Measure                        
FIELD     Operations    AMERICAS      AMERICAS_SP   ASP CSP          ASP ATT        OFFER_CREATED            0.0   
                                                                                    OPPTY_STG1_CREATED       0.0   
                                                                                    READOUTS_DONE            0.0   
                                                                                    UNCOVERED_OPPTY_VALUE    0.0   
                                                    ASP TELCO MOBILE ASP AMW        OFFER_CREATED            0.0   
...                                                                                                          ...   
TARGETING Operations    EMEAR-REGION  EMEAR_SP      SP_VDF_LG        LIBERTY_GLOBAL UNCOVERED_OPPTY_VALUE    0.0   
                                                                     VODAFONE_      OFFER_CREATED            0.0   
                                                                                    OPPTY_STG1_CREATED       0.0   
                                                                                    READOUTS_DONE            0.0   
                                                                                    UNCOVERED_OPPTY_VALUE    0.0   

Report FY                                                                                                         \
Report FQ                                                                                                          
Value Type                                                                                                Target   
Method    Measure Group Sales Level 1 Sales Level 2 Sales Level 3    Sales Level 4  Measure                        
FIELD     Operations    AMERICAS      AMERICAS_SP   ASP CSP          ASP ATT        OFFER_CREATED            0.0   
                                                                                    OPPTY_STG1_CREATED       0.0   
                                                                                    READOUTS_DONE            0.0   
                                                                                    UNCOVERED_OPPTY_VALUE    0.0   
                                                    ASP TELCO MOBILE ASP AMW        OFFER_CREATED            0.0   
...                                                                                                          ...   
TARGETING Operations    EMEAR-REGION  EMEAR_SP      SP_VDF_LG        LIBERTY_GLOBAL UNCOVERED_OPPTY_VALUE    0.0   
                                                                     VODAFONE_      OFFER_CREATED            0.0   
                                                                                    OPPTY_STG1_CREATED       0.0   
                                                                                    READOUTS_DONE            0.0   
                                                                                    UNCOVERED_OPPTY_VALUE    0.0   

Report FY                                                                                                         \
Report FQ                                                                                                     Q2   
Value Type                                                                                                Actual   
Method    Measure Group Sales Level 1 Sales Level 2 Sales Level 3    Sales Level 4  Measure                        
FIELD     Operations    AMERICAS      AMERICAS_SP   AS

In [334]:
# opp_flow_dash = pd.concat([opp_flow_dash, sub_bookings]).fillna(0.0)

In [335]:
opp_flow_dash.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 80702 entries, 0 to 797
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Method         80702 non-null  object 
 1   Measure Group  80702 non-null  object 
 2   Campaign       80702 non-null  object 
 3   Campaign Type  80702 non-null  object 
 4   Customer Name  80702 non-null  object 
 5   Sales Level 1  80702 non-null  object 
 6   Sales Level 2  80702 non-null  object 
 7   Sales Level 3  80702 non-null  object 
 8   Sales Level 4  80702 non-null  object 
 9   Offer Focus    80702 non-null  object 
 10  Value Type     80702 non-null  object 
 11  Report FY      80702 non-null  object 
 12  Report FQ      80702 non-null  object 
 13  Measure        80702 non-null  object 
 14  Value          80702 non-null  float64
 15  Unnamed: 15    80702 non-null  float64
dtypes: float64(2), object(14)
memory usage: 10.5+ MB


In [336]:
c1 = opp_flow_dash.pivot_table(index=['Method','Measure Group', 'Campaign Type','Campaign','Customer Name','Offer Focus','Sales Level 1','Sales Level 2','Sales Level 3','Sales Level 4','Measure','Report FY','Report FQ',], columns=['Value Type'], values='Value', aggfunc='sum', margins=False).fillna(0.0)  #(opp_flow_dash['Measure SubGroup'].str.contains('operations', na=False, case=False)) & 

In [337]:
c1.reset_index(inplace=True)

In [338]:
c1.loc[c1['Measure Group'].str.contains('operat', na=False, case=False)]['Offer Focus'].unique()

array(['ANY', 'SNTC', 'BCS', 'ST', 'SSPT'], dtype=object)

In [339]:
c1.to_excel(r'C:\Users\phsheari\Documents\Compass Tableau Data\Opp_Flow_Dashboard.xlsx', sheet_name = 'OpportunityFlow' , index=False)
c1.to_excel(r'C:\Users\phsheari\Documents\Compass Tableau Data\Opp_Flow_Dashboard_' + today_file + '.xlsx', sheet_name = 'OpportunityFlow_' + today_file , index=False)

In [340]:
c1['Measure'].unique()

array(['OFFER_CREATED', 'OPPTY_STG1_CREATED', 'READOUTS_DONE',
       'UNCOVERED_OPPTY_VALUE', 'Incr ARR $(000s)'], dtype=object)

In [341]:
c1.loc[(c1['Measure']=='READOUTS_DONE') & (c1['Method'].str.contains('M1 \(Coverage')) & (c1['Report FQ']=='Q2')]

Value Type,Method,Measure Group,Campaign Type,Campaign,Customer Name,Offer Focus,Sales Level 1,Sales Level 2,Sales Level 3,Sales Level 4,Measure,Report FY,Report FQ,Actual,Target
9648,M1 (Coverage Program),Operations,COVERAGE PROGRAM,COVERAGE PROGRAM - SNTC PROACTIVE ATTACH,ABDI IBRAHIM ILAC AS,SNTC,EMEAR-REGION,EMEAR_MEA,COUNTRY_TUR,COMMERCIAL_TUR,READOUTS_DONE,2022,Q2,0.0,0.0
9652,M1 (Coverage Program),Operations,COVERAGE PROGRAM,COVERAGE PROGRAM - SNTC PROACTIVE ATTACH,ABU DHABI FUTURE ENERGY COMPANY - MASDAR,SNTC,EMEAR-REGION,EMEAR_MEA,CLUSTER_GULF,COUNTRY_ARE,READOUTS_DONE,2022,Q2,1.0,0.0
9672,M1 (Coverage Program),Operations,COVERAGE PROGRAM,COVERAGE PROGRAM - SNTC PROACTIVE ATTACH,ACCOR GROUP,SNTC,EMEAR-REGION,EMEAR_MEA,CLUSTER_GULF,COUNTRY_ARE,READOUTS_DONE,2022,Q2,1.0,0.0
9696,M1 (Coverage Program),Operations,COVERAGE PROGRAM,COVERAGE PROGRAM - SNTC PROACTIVE ATTACH,ADIX,SNTC,EMEAR-REGION,EMEAR-NORTH,COUNTRY_NETHERLANDS,COMMERCIAL_NLD,READOUTS_DONE,2022,Q2,0.0,0.0
9732,M1 (Coverage Program),Operations,COVERAGE PROGRAM,COVERAGE PROGRAM - SNTC PROACTIVE ATTACH,AGFA - GEVAERT,SNTC,EMEAR-REGION,EMEAR-NORTH,CLUSTER_BELUX,COMMERCIAL_BEL,READOUTS_DONE,2022,Q2,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37684,M1 (Coverage Program),Operations,TARGETING,SUCCESS TRACKS FOR CAMPUS TARGETING - DYNAMIC,VERMEER CORPORATION (PRIMARY),ST,AMERICAS,US COMMERCIAL,COMMERCIAL CENTRAL AREA,HEARTLAND-GULF COMMERCIAL OPERATION,READOUTS_DONE,2022,Q2,1.0,0.0
37688,M1 (Coverage Program),Operations,TARGETING,SUCCESS TRACKS FOR CAMPUS TARGETING - DYNAMIC,VOCERA COMMUNICATIONS INC,ST,AMERICAS,US COMMERCIAL,COMMERCIAL WEST AREA,GOLDEN WEST COMMERCIAL OPERATION,READOUTS_DONE,2022,Q2,1.0,0.0
37699,M1 (Coverage Program),Operations,TELEMETRY,COVERAGE PROGRAM - TELEMETRY,0.0,SNTC,AMERICAS,0.0,0.0,0.0,READOUTS_DONE,2022,Q2,0.0,435.0
37715,M1 (Coverage Program),Operations,TELEMETRY,COVERAGE PROGRAM - TELEMETRY,0.0,SNTC,APJC,0.0,0.0,0.0,READOUTS_DONE,2022,Q2,0.0,0.0


### Add Measure Group, Measure SubGroup, Offer Focus, to bookings in order to create a table like the agg_report which can be vertically stacked with the Target file data

In [342]:
opp_flow_dash.loc[opp_flow_dash['Value Type']=='Actual']['Measure Group'] #.unique()

0        Operations
1        Operations
2        Operations
3        Operations
4        Operations
           ...     
793    Bookings-New
794    Bookings-New
795    Bookings-New
796    Bookings-New
797    Bookings-New
Name: Measure Group, Length: 80318, dtype: object

In [343]:
# df[['Request ID','Created Date','Customer Name','Campaign','Alert Below 25', 'Requester Name','Requester Function','SSP','Distribution List','RR_internal', 'SAV ID', 
#     'Compass Advisor', 'Customer Identifier','Deal ID','Declined Reason','Email Sent to SSX','Method','Measure Group','Readout Date','Next Steps', 'Items Recommended',
#     'LPM Plus Assigned','LPM Plus Engaged','LPM Plus Scheduled','LPM Plus Unassigned','LPM Readouts Declined','LPM Readouts Done',
#     'Offer Created','Oppty Stg1 Created','Readouts Done','SNTC Estimate','Uncovered Oppty Value','Partner Name','Readout Date',
#     'Readout Delivery Status', 'Report FY','Report FQ','Sales Level 1','Sales Level 2','Sales Level 3', 'Sales Level 4','WORKING STATUS']]

In [344]:
# Done

In [345]:
# ['OPPTY_STG1_CREATED','DATA_PACKAGE_CREATED','READOUTS_DONE','OFFER_CREATED']

In [346]:
df.to_pickle(excel_file_path_sfdc_version)
df.to_excel(excel_file_path_tracker,sheet_name='TrackerData_' + today_file, index=False)
df.to_excel(excel_file_path_tableau,sheet_name='TableauData_' + today_file, index=False)

C:\Users\phsheari\.conda\envs\compass_env\lib\site-packages\xlsxwriter\worksheet.py:986: UserWarning: Ignoring URL 'https://cisco.sharepoint.com/:p:/r/sites/ONEIB/Shared%20Documents/Compass%20Operations/Compass%20PPTs/Americas/Commercial/Compass_Analysis_BOSSELMAN_INC_111711_202201.pptx?d=w87f0671f4d61459f9509bebefc13b3d1&csf=1&web=1&e=itouom' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  warn("Ignoring URL '%s' since it exceeds Excel's limit of "
C:\Users\phsheari\.conda\envs\compass_env\lib\site-packages\xlsxwriter\worksheet.py:986: UserWarning: Ignoring URL 'https://cisco.sharepoint.com/:p:/r/sites/ONEIB/Shared%20Documents/Compass%20Operations/Compass%20PPTs/Americas/Commercial/Compass_Analysis_RIDGEVIEW_MEDIC_111833_202201.pptx?d=w2409b14b5bef401baf0c90c5651d9be1&csf=1&web=1&e=cdQeZ4' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  warn("Ignoring URL '%s' since it exceeds Excel's limit of "
C:\Users\phsheari\.conda\envs\compass_env\lib\site-packages\

In [347]:
finish = dt.datetime.now()
elapsed = (finish - start)
print(elapsed/60.0)

0:00:11.919197


In [348]:
df

,Request ID,Create Date,Created By,Request Type,Modified,Modified By,Date Completed,LAST_UPDATED,QA Approve,REVIEW_REQUIRED_FLAG,...,RRT_LPM+,LPM_OFFERS,Method,Measure Group,Offer Focus,Value Type,Report FY,Report FQ,Injection Month,Injection Quarter
0,118405,2022-04-07,vikantha@cisco.com,Reactive,2022-04-11,cx-compass-two.gen@cisco.com,2022-04-11,2022-04-19 13:32 PM Eastern,False,False,...,0,0,M6 (CXIB),Operations,SNTC,Actual,2022,Q3,202209,2022-Q3
1,118396,2022-04-07,vikantha@cisco.com,Reactive,2022-04-11,cx-compass-two.gen@cisco.com,2022-04-11,2022-04-19 13:32 PM Eastern,False,False,...,0,0,M6 (CXIB),Operations,SNTC,Actual,2022,Q3,202209,2022-Q3
2,118397,2022-04-07,vikantha@cisco.com,Reactive,2022-04-11,cx-compass-two.gen@cisco.com,2022-04-11,2022-04-19 13:32 PM Eastern,False,False,...,0,0,M6 (CXIB),Operations,SNTC,Actual,2022,Q3,202209,2022-Q3
3,118398,2022-04-07,vikantha@cisco.com,Reactive,2022-04-11,cx-compass-two.gen@cisco.com,2022-04-11,2022-04-19 13:32 PM Eastern,False,False,...,0,0,M6 (CXIB),Operations,SNTC,Actual,2022,Q3,202209,2022-Q3
4,118399,2022-04-07,vikantha@cisco.com,Reactive,2022-04-11,cx-compass-two.gen@cisco.com,2022-04-11,2022-04-19 13:32 PM Eastern,False,False,...,0,0,M6 (CXIB),Operations,SNTC,Actual,2022,Q3,202209,2022-Q3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42650,114327,2022-03-03,vikantha@cisco.com,Targeting Campaign,2022-04-14,ggomezbe@cisco.com,2022-03-08,2022-04-19 13:32 PM Eastern,False,False,...,0,0,M1 (Coverage Program),Operations,SNTC,Actual,2022,Q3,202208,2022-Q3
42651,114365,2022-03-03,vikantha@cisco.com,Targeting Campaign,2022-04-14,ggomezbe@cisco.com,2022-03-08,2022-04-19 13:32 PM Eastern,False,False,...,0,0,M1 (Coverage Program),Operations,SNTC,Actual,2022,Q3,202208,2022-Q3
42652,114344,2022-03-03,vikantha@cisco.com,Targeting Campaign,2022-04-14,ggomezbe@cisco.com,2022-03-08,2022-04-19 13:32 PM Eastern,False,False,...,0,0,M1 (Coverage Program),Operations,SNTC,Actual,2022,Q3,202208,2022-Q3
42653,114370,2022-03-03,vikantha@cisco.com,Targeting Campaign,2022-04-14,ggomezbe@cisco.com,2022-03-08,2022-04-19 13:32 PM Eastern,False,False,...,0,0,M1 (Coverage Program),Operations,SNTC,Actual,2022,Q3,202208,2022-Q3


In [349]:
df.to_pickle(r'D:\Compass Trackers\TrackerData.pkl')
df.to_excel(r'D:\Compass Trackers\Tracker Data_' + today_file + '.xlsx', index=False)
# df.to_csv(hyper_csv_file_path_tracker, index=False)

FileNotFoundError: [Errno 2] No such file or directory: 'D:\\Compass Trackers\\TrackerData.pkl'

In [ ]:
df.loc[(df['Method'].str.contains('M1', na=False, case=False)) & (df['ALERT_BELOW_25'] == 1)][['Create Date','Customer Name','Compass Advisor','Readout Delivery Status','LPM_PLUS_ENGAGED','ADVISOR_NOTES','LPM_NOTES']]

In [ ]:
Done

In [ ]:
len(df.loc[df['Request ID'].isnull()]['Request ID'].unique())

In [ ]:
# df.loc[df['Request ID'].isin(['202120216643','202120216987'])][['Request ID','Requester Name','Create Date','GUID','CR Party ID','SAV ID','Customer Name','Lvl1','Lvl2 (Region)','Compass Campaign Name']]

In [ ]:
today = dt.datetime.today()

In [ ]:
today

### Create the COMPASS GENERATED file for Tableau

In [ ]:
compass_generated = df.loc[(df['Request FY'].isin(['2021','2022'])) & (df['DELIVERABILITY']=='Good') & (df['REMOVE_FROM_COUNT'] == True)][['Request ID','Create Date','Customer Name','GUID','CR Party ID', 'CR Party Name', 'CAV BU ID', 'CAV ID', 
                                                                                                                                'Deal ID', 'SAV ID', 'Contract ID', 'Inventory Name', 'Appliance ID', 'Requester Name','Requester Function','Campaign','Partner Name', 'Partner Deal','Status', 
                                                                                                                                'Lvl1', 'Lvl2 (Region)', 'Lvl3','Request Type', 'Date Completed', 'Compass Advisor','Readout_Date','Readout Delivery Status', 'Items Recommended','HashedReadout','CONCIERGE_ADVISOR_ASSIGNED',
                                                                                                                                'CONCIERGE_READOUT_DATE', 'CONCIERGE_READOUT_RESULT', 'Report Group', 'Request FY', 'Request FH','Request FQ', 'Request FM','Request FWOFQ','Request FWOFY','Readout FY','Readout FH','Readout FQ','Readout FM', 'Readout FWOFM', 'Readout FWOFQ', 'Readout FWOFY', 
                                                                                                                                'Recipient Confirmation','RR (internal)','is_Targeting_Campaign', 'Campaign Type', 'Targeting Campaign Type', 'REMOVE_FROM_COUNT',]]#.pivot_table(values='Customer Name', columns='Deliverability', aggfunc='count')

In [ ]:
compass_generated.columns

In [ ]:
compass_generated.to_excel(r'C:\Users\phsheari\Documents\Compass Tableau Data\CompassGenerated.xlsx', sheet_name='CompassGenerated',  index=False)

### Readouts to Publish to Tableau

In [ ]:
readouts_for_tableau_path = r'C:\Users\phsheari\Documents\Compass Tableau Data\Readouts.xlsx'
undelivered_readouts_for_tableau_path = r'C:\Users\phsheari\Documents\Compass Tableau Data\UndeliveredReadouts.xlsx'

In [ ]:
_11months_ago = today + dt.timedelta(days = -330)

In [ ]:
# df['Date Readout Done'] = pd.to_datetime(df['Date Readout Done'])

In [ ]:
# readouts = df.loc[(df['Date Readout Done'].notnull()) & (df['Readout Delivery Status'].notnull())][['Request ID', 'Sheet Name', 'Previous Request ID', 'Create Date', 'Customer Name', 'GUID', 'CR Party ID','CAV ID', 'Deal ID', 'Requester Name',
#                                                                                                    'Partner Name', 'Compass Campaign Name','Status', 'Lvl1', 'Lvl2 (Region)', 'Lvl3','COMPASS URL 1.0', 'COMPASS PPT URL','Compass Concierge Requested',
#                                                                                                    'RR (internal)', 'Compass Advisor', 'BDM Notes', 'Items Recommended', 'Date Readout Done', 'Readout Delivery Status', 'Reason Declined', 'EXT_BDM_ASSIGNED',
#                                                                                                    'EXT_DATE_READOUT_DONE', 'EXT_READOUT_RESULT', 'Forecast Stage','Projected Booking ($,000)','Target Fiscal Month', 'Target Fiscal Quarter', 'Target Fiscal Year',
#                                                                                                    'Deliverability','Recipient Confirmation', 'Request FY', 'Request FQ', 'Request FM', 'Request FWOFM', 'Request FWOFQ', 'Request FWOFY', 'Readout FY', 
#                                                                                                    'Readout FQ','Readout FM', 'Readout FWOFM', 'Readout FWOFQ', 'Readout FWOFY', 'Campaign Type','IR_BCS', 'IR_EC', 'IR_DNAC', 'IR_SSPT', 'IR_ST', 'IR_SNTC', 
#                                                                                                    'IR_IRR', 'IR_PR', 'IR_BRW', 'TC_Priority_Recommend','Squeezed','Prepend']]

In [ ]:
# readouts['Prehash_Entity'] = readouts.apply(lambda x: x['Prepend'] + x['FakeID'], axis=1)

In [ ]:
# readouts[['Prehash_Entity','GUID','CAV ID','Customer Name']]

In [ ]:
df.loc[:,df.columns.str.contains('COMPASS URL', na=False, case=False)]

In [ ]:
readouts_publish_to_tableau  = df.loc[(df['Date Readout Done'].notnull()) & 
                                 ~df['Readout Delivery Status'].isin(['Declined','Postponed']) & 
                                 (df['GUID'].notnull() | df['CAV ID'].notnull()) &
                                 (df['Date Readout Done'] <= today) & 
                                 (df['REMOVE_FROM_COUNT'] == True) &
                                 (df['DELIVERABILITY']=='Good')][['Request ID', 'Create Date', 'Customer Name', 'GUID', 'CR Party ID','CAV ID', 'SAV ID','Deal ID', 'Requester Name',
                                                                  'Partner Name','Partner Deal', 'Campaign', 'Lvl1', 'Lvl2 (Region)', 'Lvl3','COMPASS URL', 'COMPASS PPT URL','RR (internal)', 'Report Group', 'Compass Advisor', 'Items Recommended', 'Date Readout Done', 'Readout Delivery Status', 
                                                                  'CONCIERGE_ADVISOR_ASSIGNED', 'CONCIERGE_READOUT_DATE', 'CONCIERGE_READOUT_RESULT', 'Forecast Stage','Opportunity Value','Target Fiscal Month', 'Target Fiscal Quarter', 'Target Fiscal Year', 'TC_Priority_Recommend', 
                                                                  'Recipient Confirmation', 'Request FY', 'Request FH','Request FQ', 'Request FM', 'Request FWOFM', 'Request FWOFQ', 'Request FWOFY', 'Readout FY','Readout FH','Readout FQ','Readout FM', 'Readout FWOFM', 'Readout FWOFQ', 'Readout FWOFY', 
                                                                  'is_Targeting_Campaign', 'Campaign Type','Targeting Campaign Type','Prehash_Entity','Entity_to_hash','HashedReadout','FakeID','REMOVE_FROM_COUNT']]

In [ ]:
def item_recommended_count(item_name, items_recommended):
    if pd.isnull(items_recommended):
        return 0
    elif item_name in items_recommended:
        return 1
    else:
        return 0

In [ ]:
def bcs_recommended_count(items_recommended):
    if pd.isnull(items_recommended):
        return 0
    elif 'BCS' in items_recommended:
        return 1
    else:
        return 0

In [ ]:
def ec_recommended_count(items_recommended):
    if pd.isnull(items_recommended):
        return 0
    elif 'EC' in items_recommended:
        return 1
    else:
        return 0

In [ ]:
def irr_recommended_count(items_recommended):
    if pd.isnull(items_recommended):
        return 0
    elif 'IRR' in items_recommended:
        return 1
    else:
        return 0

In [ ]:
def dnac_recommended_count(items_recommended):
    if pd.isnull(items_recommended):
        return 0
    elif 'DNA' in items_recommended:
        return 1
    else:
        return 0

In [ ]:
def sspt_recommended_count(items_recommended):
    if pd.isnull(items_recommended):
        return 0
    elif 'SSPT' in items_recommended:
        return 1
    else:
        return 0

In [ ]:
def sntc_recommended_count(items_recommended):
    if pd.isnull(items_recommended):
        return 0
    elif 'SNTC' in items_recommended:
        return 1
    else:
        return 0

In [ ]:
def st_recommended_count(items_recommended):
    if pd.isnull(items_recommended):
        return 0
    elif 'ST' in items_recommended:
        return 1
    else:
        return 0

In [ ]:
def pr_recommended_count(items_recommended):
    if pd.isnull(items_recommended):
        return 0
    elif 'PR' in items_recommended:
        return 1
    else:
        return 0

In [ ]:
def brw_recommended_count(items_recommended):
    if pd.isnull(items_recommended):
        return 0
    elif 'BRW' in items_recommended:
        return 1
    else:
        return 0

In [ ]:
def NULL_recommended_count(items_recommended):
    if pd.isnull(items_recommended):
        return 1
    else:
        return 0

In [ ]:
# readouts_publish_to_tableau['HashEntity'] = readouts_publish_to_tableau.apply(lambda x: make_hash_entity(x['Prehash_Entity'], x['Compass Advisor'],x['Date Readout Done'], x['Items Recommended']), axis=1) #x['COMPASS URL 1.0'], 

In [ ]:
readouts_publish_to_tableau['Targeting Campaign Type'].unique()

In [ ]:
# readouts_publish_to_tableau['HashedReadout'] = readouts_publish_to_tableau.apply(lambda x: hash_the_readout(x['HashEntity']), axis=1 )

In [ ]:
# readouts_publish_to_tableau.HashedReadout.nunique()

In [ ]:
readouts_publish_to_tableau['IRC_BCS']  = readouts_publish_to_tableau['Items Recommended'].apply(bcs_recommended_count)
readouts_publish_to_tableau['IRC_EC']   = readouts_publish_to_tableau['Items Recommended'].apply(ec_recommended_count)
readouts_publish_to_tableau['IRC_IRR']  = readouts_publish_to_tableau['Items Recommended'].apply(irr_recommended_count)
readouts_publish_to_tableau['IRC_DNAC'] = readouts_publish_to_tableau['Items Recommended'].apply(dnac_recommended_count)
readouts_publish_to_tableau['IRC_SSPT'] = readouts_publish_to_tableau['Items Recommended'].apply(sspt_recommended_count)
readouts_publish_to_tableau['IRC_ST']   = readouts_publish_to_tableau['Items Recommended'].apply(st_recommended_count)
readouts_publish_to_tableau['IRC_SNTC'] = readouts_publish_to_tableau['Items Recommended'].apply(sntc_recommended_count)
readouts_publish_to_tableau['IRC_PR']   = readouts_publish_to_tableau['Items Recommended'].apply(pr_recommended_count)
readouts_publish_to_tableau['IRC_BRW']  = readouts_publish_to_tableau['Items Recommended'].apply(brw_recommended_count)
readouts_publish_to_tableau['IRC_NULL'] = readouts_publish_to_tableau['Items Recommended'].apply(NULL_recommended_count)

In [ ]:
readouts_publish_to_tableau.to_excel(readouts_for_tableau_path,sheet_name='TableauReadoutData', index=False)
readouts_publish_to_tableau.to_excel(r'C:\Users\phsheari\Documents\Compass Readouts\Compass Readouts.xlsx',sheet_name='TableauReadoutData', index=False)

In [ ]:
readouts_publish_to_tableau.groupby('REMOVE_FROM_COUNT')['Request ID'].count()

In [ ]:
latam_readouts1 = readouts_publish_to_tableau['Lvl2 (Region)']=='LATIN AMERICA'
latam_readouts3 = readouts_publish_to_tableau['Date Readout Done'] <= '2021-05-07'
latam_readouts2 = readouts_publish_to_tableau['Date Readout Done'] > '2020-10-07'

In [ ]:
latam_readouts = readouts_publish_to_tableau.loc[(latam_readouts1 & latam_readouts2 & latam_readouts3)][['Request ID','Date Readout Done','Customer Name','GUID','Compass Advisor','Readout Delivery Status','IRC_BCS','IRC_SSPT','IRC_EC','IRC_ST','IRC_SNTC']]

In [ ]:
#latam_readouts.to_excel(r'C:\Users\phsheari\Documents\FIN BI Data\1_AMERICAS\FY21Q4 Data\Latam Readouts.xlsx',index=False)

In [ ]:
readouts_publish_to_tableau.loc[readouts_publish_to_tableau['Customer Name'].str.contains('kfh', na=False, case=False)]

In [ ]:
readouts_not_done_condition1 = ((df['Readout Delivery Status'].isnull()) | (df['Readout Delivery Status'].isin(['Declined','Postponed']))) & (df['Items Recommended'].isnull())

In [ ]:
readouts_not_done_condition2 = df['DELIVERABILITY']=='Good'

In [ ]:
readouts_not_done_condition3 = ~(df['Request FY']==2021 & df['Campaign'].str.contains('growth initiative', case=False))

In [ ]:
readouts_not_done_condition4 = df['Request FY'].isin(['2021','2022'])

In [ ]:
readouts_not_done_condition5 = df['REMOVE_FROM_COUNT']==True

In [ ]:
readouts_not_done = df.loc[readouts_not_done_condition1 & readouts_not_done_condition2 & readouts_not_done_condition3 & readouts_not_done_condition4 & readouts_not_done_condition5][['Request ID', 'Create Date', 'Customer Name', 'GUID', 'CR Party ID','CAV ID', 'Deal ID','Requester Name',
                                                                                                   'Partner Name','Partner Deal', 'Campaign','Status', 'Lvl1', 'Lvl2 (Region)', 'Lvl3','COMPASS URL', 'COMPASS PPT URL','Compass Concierge Requested',
                                                                                                   'RR (internal)','Report Group','Compass Advisor', 'BDM Notes', 'Items Recommended', 'Date Readout Done', 'Readout Delivery Status', 'Reason Declined', 'CONCIERGE_ADVISOR_ASSIGNED',
                                                                                                   'CONCIERGE_READOUT_DATE', 'CONCIERGE_READOUT_RESULT',  'Forecast Stage','Opportunity Value','Target Fiscal Month', 'Target Fiscal Quarter', 'Target Fiscal Year',
                                                                                                   'DELIVERABILITY','Recipient Confirmation', 'Request FY', 'Request FH', 'Request FQ', 'Request FM', 'Request FWOFM', 'Request FWOFQ', 'Request FWOFY', 'Readout FY','Readout FH', 
                                                                                                   'Readout FQ','Readout FM', 'Readout FWOFM', 'Readout FWOFQ', 'Readout FWOFY', 'is_Targeting_Campaign', 'Campaign Type', 'Targeting Campaign Type','IR_BCS', 'IR_EC', 'IR_DNAC', 'IR_SSPT', 'IR_ST', 'IR_SNTC', 
                                                                                                   'IR_IRR', 'IR_PR', 'IR_BRW', 'TC_Priority_Recommend','Entity_to_hash','HashedReadout','REMOVE_FROM_COUNT']]

In [ ]:
readouts_not_done

In [ ]:
### Readouts Scheduled
readouts_not_done.loc[(readouts_not_done['Date Readout Done']>= today) & (readouts_not_done['Request FY']>=str(today.year - 1))]

In [ ]:
### Readouts Not Done/ Not Scheduled / Not Declined
readouts_not_done.loc[(readouts_not_done['Date Readout Done'].isnull()) & (readouts_not_done['Request FY']>=str(today.year-1)) & ~(readouts_not_done['Readout Delivery Status'].isin(['Declined','Postponed']))].groupby(['Request FY','Campaign','Request FQ', 'Request FM',]).count()[['Request ID']]

In [ ]:
### Readouts Declined or Postponed
readouts_not_done.loc[readouts_not_done['Readout Delivery Status'].isin(['Declined','Postponed'])]

In [ ]:
readouts_not_done.to_excel(undelivered_readouts_for_tableau_path,sheet_name='Undelivered', index=False)
readouts_not_done.to_excel(r'C:\Users\phsheari\Documents\Compass Readouts\Undelivered Readouts.xlsx',sheet_name='Undelivered', index=False)

In [ ]:
readouts_not_done.columns

## Compass Partner Data

#### ---Partner Compass Delivered

In [ ]:
CURRENT_FISCAL_YEAR = today.year

In [ ]:
CURRENT_FISCAL_YEAR   #& (df['Request FY']==CURRENT_FISCAL_YEAR)   ##  ((df['Deal ID']==-44444) | df['Sheet Name'].str.contains('partner', case=False)) & 

In [ ]:
partner_model = (df['Deal ID']==-44444) & (df['Request FY'] == str(CURRENT_FISCAL_YEAR))  #[['Request ID','Request FY']]

In [ ]:
partner_compass_delivered = df.loc[partner_model][['Request ID','Create Date','Customer Name','GUID','Partner Name', 'Partner Deal', 'Lvl1','Lvl2 (Region)','Lvl3','Report Group','Compass Advisor','Date Readout Done', 'Request FY',  'Request FH', 
                                                   'Request FQ','Request FM', 'Request FWOFM', 'Request FWOFQ', 'Request FWOFY','Readout FY', 'Readout FH', 'Readout FQ','Readout FM', 'Readout FWOFM', 'Readout FWOFQ', 'Readout FWOFY']] 

In [ ]:
# partner_compass_delivered['GUID'] = partner_compass_delivered['GUID'].astype(int)

In [ ]:
partner_compass_delivered.sort_values(by=(['Lvl1','Partner Name','Lvl2 (Region)','Create Date','Request ID','Customer Name']), inplace=True)

In [ ]:
# partner_compass_delivered.groupby(['Request FY','Lvl1','Partner Name']).count()[['Request ID','Compass Advisor','Date Readout Done']]

#### ---Partner Readouts Delivered

In [ ]:
partner_readouts_delivered = df.loc[(df['Deal ID']==-44444) & 
#                                     (df['Request FY']==CURRENT_FISCAL_YEAR) & 
                                    (df['Readout Delivery Status'].str.contains('complete', case=False))  & 
                                    (df['DELIVERABILITY']=='Good')][['Request ID','Create Date','Customer Name','GUID','Partner Name','Partner Deal','Lvl1','Lvl2 (Region)','Compass Advisor','Date Readout Done','Readout Delivery Status','Items Recommended','Readout FY','Readout FH','Readout FQ','Readout FM',]]

In [ ]:
partner_readouts_delivered['GUID'] = partner_readouts_delivered['GUID'].astype(int)

In [ ]:
partner_readouts_delivered.sort_values(by=(['Lvl1','Partner Name','Create Date','Request ID','Customer Name']), inplace=True)

In [ ]:
partner_readouts_delivered.groupby(['Readout FY','Readout FQ','Readout FM']).count()['Request ID']

In [ ]:
partner_compass_delivered.to_excel(r'C:\Users\phsheari\Documents\Compass Tableau Data\PartnerData.xlsx',sheet_name='Partners', index=False)

In [ ]:
Done

## Compass Pipeline Data

In [ ]:
### Compass Pipeline Conditions
forecast_stages = ['1 - Prospect','2 - Qualification','3 - Proposal','4 - Agreement']
pipeline1 = (df['Forecast Stage'].isin(forecast_stages))
pipeline2 = (df['Deliverability']=='Good')
pipeline3 = ((df['Target Fiscal Year'] == str(CURRENT_FISCAL_YEAR)) | (df['Target Fiscal Year'] == str(CURRENT_FISCAL_YEAR + 1)))
pipeline4 = (df['Request FY'].isin([str(CURRENT_FISCAL_YEAR - 1),str(CURRENT_FISCAL_YEAR)]))

In [ ]:
compass_pipeline_data = df.loc[pipeline1 & pipeline2 & pipeline3 & pipeline4][['Request ID','Create Date','Customer Name','Requester Name','GUID','CAV ID','Deal ID', 'Lvl1','Lvl2 (Region)','Forecast Stage','Forecast Status','Opportunity Value','Target Fiscal Month','Target Fiscal Quarter','Target Fiscal Year']] 

In [ ]:
compass_pipeline_data['Deal ID'] = compass_pipeline_data['Deal ID'].astype(int)

In [ ]:
# compass_pipeline_data['GUID'] = compass_pipeline_data['GUID'].astype(int)

In [ ]:
compass_pipeline_data.loc[compass_pipeline_data['GUID'].isnull()]

In [ ]:
pd.pivot_table(data=compass_pipeline_data, values='Projected Booking ($,000)', index=['Target Fiscal Year','Target Fiscal Quarter'], columns='Forecast Stage', aggfunc=['sum','count'], fill_value = 0.0)

In [ ]:
compass_pipeline_data.to_excel(r'C:\Users\phsheari\Documents\Compass Tableau Data\CompassPipeline.xlsx',sheet_name='Pipeline', index=False)

In [ ]:
non_reporting_campaigns = ['accelerator','test','test request','proactive - ignore']

In [ ]:
## Compass Inventory Generated
compass_generated_inventory = df.loc[(df['Deliverability']=='Good') & ~(df['Campaign'].str.lower().isin(non_reporting_campaigns))][['Request ID', 'Create Date','Date Completed','Complete?','Date Readout Done','Customer Name','Requester Name','Requester Function','Partner Name','Partner Deal','Recipient Confirmation','Compass Campaign Name', 'Request FY', 'Lvl1','Lvl2 (Region)','Lvl3','Campaign Type','Record Type','REMOVE_FROM_COUNT','RR (internal)','Targeting Campaign Type']]

In [ ]:
compass_generated_inventory.loc[(compass_generated_inventory['Request FY']=='2021') & (compass_generated_inventory['REMOVE_FROM_COUNT']==True) & ~(compass_generated_inventory['RR (internal)']=='NO')]  #(compass_generated_inventory['Recipient Confirmation']=='Confirmed') & 

In [ ]:
compass_generated_inventory.to_excel(r'C:\Users\phsheari\Documents\Compass Tableau Data\Compass_Generated_Inventory.xlsx', sheet_name='Compass Generated', index=False)

In [ ]:
# if finish.weekday() == 4:
#     send_file = r'C:\Users\phsheari\Documents\Compass Pipeline' + "\\" + 'Compass_Maven_Bookings_Pipeline_Partners_ETL_File_' + today_file[:8] + '.xlsx'
#     with pd.ExcelWriter(send_file) as writer:  #endfilename
#         compass_pipeline_data.to_excel(writer, sheet_name = "Compass Pipeline Data", index=False)
#         compas_finbi_bookings.to_excel(writer, sheet_name = 'Compass FINBI Bookings', index=False)
#         partner_compass_delivered.to_excel(writer, sheet_name = "Partner Compass Delivered", index=False)
#         partner_readouts_delivered.to_excel(writer, sheet_name = "Partner Readouts Delivered", index=False)
#         finbi_partner_deals.to_excel(writer, sheet_name = 'Partner Bookings (from Compass)', index = False)   

### LATAM records from Smartsheet

In [ ]:
df.loc[(df['Lvl2 (Region)'].notnull()) & 
       (df['Lvl2 (Region)'].str.contains('LATIN AMERICA', case=False)) & 
       (df['Request FY'] == '2021')  & 
       ~(df['Campaign'].str.contains('growth', na=False, case=False))
      ].groupby(['Campaign Type','Targeting Campaign Type']).count()[['Request ID','Compass Advisor','Date Readout Done','Readout Delivery Status','Items Recommended','HashedReadout']]

In [ ]:
finish = dt.datetime.now()
elapsed = (finish - start)
print(elapsed/60.0)

In [ ]:
Done

## Healthcare records exercise

In [ ]:
h1 = df['Customer Name'].str.contains('medical', case=False)

In [ ]:
h2 = df['Customer Name'].str.contains('hospital', case=False)

In [ ]:
h3 = df['Customer Name'].str.contains('health', case=False)

In [ ]:
h4 = df['Customer Name'].str.contains('physician', case=False)

In [ ]:
h5 = df['Customer Name'].str.contains('pharma', case=False)

In [ ]:
h6 = df['Customer Name'].str.contains('laboratori', case=False)

In [ ]:
h7 = df['Customer Name'].str.contains('labs', case=False)

In [ ]:
h8 = df['Customer Name'].str.contains('pfizer', case=False)

In [ ]:
h9 = df['Customer Name'].str.contains('bayer', case=False)

In [ ]:
h10 = df['Customer Name'].str.contains('merck', case=False)

In [ ]:
h11 = df['Customer Name'].str.contains('roche', case=False)

In [ ]:
h12 = df['Customer Name'].str.contains('novartis', case=False)

In [ ]:
h13 = df['Customer Name'].str.contains('glaxo', case=False)

In [ ]:
h14 = df['Customer Name'].str.contains('johnson & johnson', case=False)

In [ ]:
h15 = df['Customer Name'].str.contains('sanofi', case=False)

In [ ]:
h16 = df['Customer Name'].str.contains('abbvie', case=False)

In [ ]:
h17 = df['Customer Name'].str.contains('zeneca', case=False)

In [ ]:
h18 = df['Customer Name'].str.contains('baxter', case=False)

In [ ]:
h19 = df['Customer Name'].str.contains('bausch', case=False)

In [ ]:
h20 = df['Customer Name'].str.contains('biogen', case=False)

In [ ]:
h21 = df['Customer Name'].str.contains('celgene', case=False)

In [ ]:
h22 = df['Customer Name'].str.contains('moderna', case=False)

In [ ]:
h23 = df['Customer Name'].str.contains('stryker', case=False)

In [ ]:
h24 = df['Customer Name'].str.contains('teva', case=False)

In [ ]:
h25 = df['Customer Name'].str.contains('medtronic', case=False)

In [ ]:
h26 = df['Customer Name'].str.contains('diagnostic', case=False)

In [ ]:
today = dt.datetime.now()

In [ ]:
last_90_days = today.date() + dt.timedelta(days=-90)

In [ ]:
last_90_days

In [ ]:
h_last_90 = (df['Create Date'] >= last_90_days)

In [ ]:
health2 = df.loc[(h1 | h2 | h3 | h4 | h5 | h6 | h8 | h9 | h10 | h11 | h12 | h13 | h14 | h15 | h16 | h17 | h18 | h19 | h20 | h21 | h22 | h23 | h24 | h25 | h26) & h_last_90][['Request ID','Create Date','Customer Name','GUID','Lvl1','Lvl2 (Region)','Compass Advisor','Date Readout Done', 'Readout Delivery Status']]

In [ ]:
health2.shape

In [ ]:
health2.sort_values(by=(['Create Date','Customer Name']), inplace=True)

In [ ]:
health2.to_excel(r'C:\Users\phsheari\Documents\Francisco\Healthcare_Compass_Accounts3.xlsx', index=False)

In [ ]:
emma_scudder = df.loc[h_last_90 & (df['Deliverability']=='Good')][['Request ID','Create Date','Customer Name','GUID', 'CAV ID','CAV BU ID','SAV ID','CR Party ID','CR Party Name','Appliance ID','Contract ID', 'Lvl1','Lvl2 (Region)','Lvl3']]

In [ ]:
emma_scudder.to_excel(r'C:\Users\phsheari\Documents\Francisco\Emma_Scudder_Request_Last_90_Days_Compass.xlsx', index=False)

### Main Work Complete

In [ ]:
df[['Request ID','Sheet Name']].nunique()

In [ ]:
finish

In [ ]:
df['Deal ID'] = df['Deal ID'].apply(convert_nan_to_int_to_str)

In [ ]:
campaign_counts = pd.DataFrame(df.loc[df['Campaign].notnull()][['Lvl1','Lvl2 (Region)','Lvl3','Campaign','Requester Name','Create Date','Request FY','Request FQ']].value_counts())

In [ ]:
campaign_counts.rename(columns={0:'Counts'}, inplace=True)

In [ ]:
campaign_counts.sort_values(['Campaign', 'Request FY','Request FQ'], inplace=True)

In [ ]:
campaign_counts.reset_index(inplace=True)

In [ ]:
campaign_counts.columns

In [ ]:
campaign_counts.to_csv(r'C:\Users\phsheari\Desktop\campaigns.csv', index=False)

In [ ]:
# bigdict = pre_dict.to_dict('records')

In [ ]:
# bigdict

In [ ]:
# dic = {}
# for x in bigdict:
#     dic[x['Request ID']] = x['Requester Name']

In [ ]:
# dic

In [ ]:
# len(dic.items()) #['Compass Campaign Name']

In [ ]:
today = dt.datetime.today().date()

In [ ]:
today

In [ ]:
_30days_ago = today + dt.timedelta(days=-30)

In [ ]:
_90days_ago = today + dt.timedelta(days=-90)

In [ ]:
_90days_ago

In [ ]:
cond0 = df['Compass Campaign Name'].notnull()
cond11 = df['Compass Campaign Name'].str.contains('target', case=False)
cond12 = df['Compass Campaign Name'].str.contains('success', case=False)
cond13 = df['Compass Campaign Name'].str.contains('bcs', case=False)
cond3 = df['Date Readout Done'] >= _30days_ago
cond4 = (df['Date Readout Done'] > _90days_ago) & (df['Date Readout Done'] <= _30days_ago)

In [ ]:
cond5 =  df['Create Date'] >= _30days_ago
cond6 = df['Date Readout Done'].isnull()
cond7 = (df['RR (internal)'].isnull() | df['RR (internal)'].str.lower()=='yes')
cond8 = (df['RR (internal)'].isnull() | df['RR (internal)'].str.lower()=='no')

In [ ]:
_30_day_data = df.loc[cond3 & cond7][['Request ID', 'Create Date', 'Sheet Name','Requester Name','Customer Name','GUID','Compass Campaign Name','Compass Advisor','Date Readout Done', 'Readout Delivery Status']]

In [ ]:
new_last_30_days = df.loc[cond0 & (cond11 | cond12 | cond13) & cond5 & cond6][['Request ID', 'Create Date', 'Sheet Name','Requester Name','Customer Name','GUID','Compass Campaign Name','Compass Advisor','Date Readout Done', 'Readout Delivery Status']]

In [ ]:
new_entries_to_cancel = pd.merge(new_last_30_days, _30_day_data, how ='inner', left_on = ['GUID','Customer Name'], right_on = ['GUID','Customer Name'])[['Request ID_x', 'Create Date_x', 'Sheet Name_x','Requester Name_x','Customer Name','GUID','Compass Campaign Name_x','Compass Advisor_x','Date Readout Done_x', 'Readout Delivery Status_x']]

In [ ]:
newcols = []
for col in new_entries_to_cancel.columns:
    newcols.append(col.replace('_x',''))
new_entries_to_cancel.columns = newcols

In [ ]:
new_entries_to_cancel.drop_duplicates(inplace=True)

In [ ]:
new_entries_to_cancel.sort_values(by=['Sheet Name','Request ID'], inplace=True, ignore_index=True)

In [ ]:
new_entries_to_cancel['Requester Name'] = new_entries_to_cancel['Requester Name'].apply(lambda x: None if x == 'nan@cisco.com' else x)

In [ ]:
new_entries_to_cancel

In [ ]:
_90day_data = df.loc[cond0 & (cond11 | cond12 | cond13) & cond4 & cond8][['Request ID', 'Create Date', 'Sheet Name','Requester Name','Customer Name','GUID','Compass Campaign Name','Compass Advisor','Date Readout Done', 'Readout Delivery Status']]

In [ ]:
new_entries_to_schedule = pd.merge(new_last_30_days, _90day_data, how ='inner', left_on = ['GUID','Customer Name'], right_on = ['GUID','Customer Name'])[['Request ID_x', 'Create Date_x','Sheet Name_x','Requester Name_x','Customer Name','GUID','Compass Campaign Name_x','Compass Advisor_x','Date Readout Done_x', 'Readout Delivery Status_x']]

In [ ]:
newcols_sched = []
for col in new_entries_to_schedule.columns:
    newcols_sched.append(col.replace('_x',''))
new_entries_to_schedule.columns = newcols_sched

In [ ]:
new_entries_to_schedule.drop_duplicates(inplace=True)

In [ ]:
new_entries_to_schedule.sort_values(by=['Sheet Name','Request ID'], inplace=True, ignore_index=True)

In [ ]:
# new_entries_to_schedule['Requester Name'] = new_entries_to_schedule['Requester Name'].apply(lambda x: None if x.startswith('nan') else x)

In [ ]:
new_entries_to_schedule

In [ ]:
with pd.ExcelWriter(r'C:\\Users\\phsheari\\Documents\\Compass Scheduling' + "\\" + 'Stacey_To_Schedule_' + today_file[:8] + '.xlsx') as writer:  #endfilename
    new_entries_to_cancel.to_excel(writer, sheet_name = 'Entries for NO', index = False)
    new_entries_to_schedule.to_excel(writer, sheet_name = 'Entries to Schedule', index = False)

In [ ]:
pd.options.display.float_format = '{:,.2f}'.format

In [ ]:
partner_list = list(df.loc[df['Partner Name'].notnull()]['Partner Name'].str.lower()\
                                                    .replace({'. ':'',
                                                              '.':''})\
                                                    .unique()
                   )

In [ ]:
finbi_df.columns

In [ ]:
finbi_df.loc[finbi_df['Partner Deal']==True] [['Sales Level 1','Sales Level 2','GUID Name','CAV Name','Partner','Fiscal Quarter ID','ProductType','CX Product Category','TCV']]

In [ ]:
finbi_partners_booked = pd.pivot_table(finbi_df.loc[finbi_df['Partner Deal']==True][['Sales Level 1','Sales Level 2','GUID Name','CAV Name','Campaign','Partner','Fiscal Quarter ID','ProductType','CX Product Category','TCV']], index=['Sales Level 1','Sales Level 2','Partner','GUID Name','CAV Name','Campaign','ProductType','CX Product Category',], values='TCV', fill_value='999', columns=['Fiscal Quarter ID'], margins_name='Total')

In [ ]:
finbi_partners_booked.reset_index(inplace=True)

In [ ]:
finbi_partners_booked